In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LassoCV

# -----------------------------
# Load data
# -----------------------------
df = pd.read_csv(r"C:\Users\liuc\Downloads\CH_ECON_V3.csv")

TARGET = "econ_hires_per_1k"
y = pd.to_numeric(df[TARGET], errors="coerce")

mask = y.notna()
df = df.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

# -----------------------------
# Time index (for TimeSeriesSplit)
# -----------------------------
if "year" in df.columns and "quarter" in df.columns:
    q = df["quarter"].astype(str).str.replace("Q", "", regex=False)
    df["_time_index"] = pd.to_numeric(df["year"], errors="coerce") * 4 + pd.to_numeric(q, errors="coerce")
else:
    df["_time_index"] = np.arange(len(df))

df = df.sort_values("_time_index").reset_index(drop=True)

# -----------------------------
# Feature selection
# -----------------------------
drop_cols = {TARGET, "_time_index"}
categorical_cols = [c for c in ["state", "county", "industry", "year", "quarter"] if c in df.columns]
numeric_cols = [c for c in df.columns if c not in drop_cols and c not in categorical_cols]

for c in numeric_cols:
    if df[c].dtype == "object":
        df[c] = pd.to_numeric(df[c], errors="coerce")

X = df[numeric_cols + categorical_cols]

# -----------------------------
# Preprocessing + LASSO
# -----------------------------
preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), numeric_cols),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), categorical_cols)
    ],
    verbose_feature_names_out=False
)

model = Pipeline([
    ("preprocess", preprocess),
    ("lasso", LassoCV(
        cv=TimeSeriesSplit(n_splits=5),
        n_alphas=200,
        max_iter=30000,
        random_state=42
    ))
])

model.fit(X, y)

# -----------------------------
# Coefficient table (FINAL OUTPUT)
# -----------------------------
feature_names = model.named_steps["preprocess"].get_feature_names_out()
coefs = model.named_steps["lasso"].coef_

coef_table = (
    pd.DataFrame({
        "feature": feature_names,
        "coefficient": coefs
    })
    .assign(abs_coefficient=lambda d: d["coefficient"].abs())
    .sort_values("abs_coefficient", ascending=False)
    .reset_index(drop=True)
)

coef_table


c:\Users\liuc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: ['State']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


,feature,coefficient,abs_coefficient
0,econ_hire_rate,12.445247,12.445247
1,econ_emp_per_1k,10.715410,10.715410
2,STATE_% Children in Poverty,-0.513073,0.513073
3,state_avg_earnings,-0.236653,0.236653
4,STATE_% Low Birthweight,-0.208205,0.208205
5,STATE_Food Environment Index,0.164597,0.164597
6,state_hires_total,0.159182,0.159182
7,growth_emp_qoq,0.153683,0.153683
8,growth_emp_yoy,0.121825,0.121825
9,STATE_Primary Care Physicians Rate,-0.099586,0.099586


In [26]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

from merf.merf import MERF


# -----------------------------
# 1) Load
# -----------------------------

TARGET = "econ_hires_per_1k"

# -----------------------------
# 2) Ensure numeric types for Year/quarter/target/features
# -----------------------------
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df["quarter"] = pd.to_numeric(df["quarter"], errors="coerce")
df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")

# LASSO-selected features (exclude leakage)
X_cols = [
    "econ_emp_per_1k",
    "state_avg_earnings",
    "growth_emp_qoq",
    "growth_emp_yoy",
    "growth_earn_qoq",
    "STATE_% Children in Poverty",
    "STATE_% Low Birthweight",
    "STATE_Food Environment Index",
    "STATE_Primary Care Physicians Rate",
    "STATE_Social Association Rate",
    "STATE_% Smokers",
    "STATE_Preventable Hospitalization Rate",
    "STATE_Mentally Unhealthy Days",
]

LEAKY = {"econ_hire_rate", "state_hires_total"}
X_cols = [c for c in X_cols if c in df.columns and c not in LEAKY]

for c in X_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Keep rows with essential time + target
df = df.dropna(subset=["Year", "quarter", TARGET]).copy()

# -----------------------------
# 3) Reconstruct state grouping (state blocks)
#    Assumption (matches your sheet): each state starts at minYear+minQuarter.
# -----------------------------
min_year = int(df["Year"].min())
min_q = int(df["quarter"].min())

# IMPORTANT: this uses the existing row order (state blocks are contiguous in your file)
start_of_state = (df["Year"] == min_year) & (df["quarter"] == min_q)

# Force the first row to start a group even if marker fails
start_of_state.iloc[0] = True

# state_group = 0,1,2,... per block
df["state_group"] = start_of_state.cumsum() - 1

print("Derived # of state groups:", df["state_group"].nunique())
print("First 10 group sizes:\n", df["state_group"].value_counts().sort_index().head(10))

# -----------------------------
# 4) Create time index for splitting
# -----------------------------
df["time_index"] = df["Year"] * 4 + df["quarter"]

# -----------------------------
# 5) Build modeling frame
#    (Do NOT drop rows with missing X; let SimpleImputer handle it.)
# -----------------------------
df_model = df[[TARGET, "state_group", "time_index"] + X_cols].copy()
df_model = df_model.dropna(subset=[TARGET, "state_group", "time_index"]).reset_index(drop=True)

# MERF inputs
y = df_model[TARGET].values
clusters = df_model["state_group"].astype(str)  # MUST be pandas Series
Z = np.ones((len(df_model), 1))                 # random intercept only

X_raw = df_model[X_cols].values

# Preprocess X
x_pre = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
X = x_pre.fit_transform(X_raw)

# -----------------------------
# 6) Time-based split (last 20% of time_index)
# -----------------------------
df_model = df_model.sort_values("time_index").reset_index(drop=True)

split = int(len(df_model) * 0.8)
tr = np.arange(0, split)
te = np.arange(split, len(df_model))

X_tr, X_te = X[tr], X[te]
y_tr, y_te = y[tr], y[te]
Z_tr, Z_te = Z[tr], Z[te]
cl_tr, cl_te = clusters.iloc[tr], clusters.iloc[te]

# -----------------------------
# 7) Fit MERF
# -----------------------------
rf = RandomForestRegressor(
    n_estimators=600,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

merf = MERF(rf)
merf.fit(X_tr, Z_tr, cl_tr, y_tr)

# -----------------------------
# 8) Evaluate
# -----------------------------
pred_tr = merf.predict(X_tr, Z_tr, cl_tr)
pred_te = merf.predict(X_te, Z_te, cl_te)

print("\nFEATURES USED:")
print(X_cols)

print("\nTRAIN:")
print("  R2  :", r2_score(y_tr, pred_tr))
print("  RMSE:", mean_squared_error(y_tr, pred_tr, squared=False))

print("\nTEST:")
print("  R2  :", r2_score(y_te, pred_te))
print("  RMSE:", mean_squared_error(y_te, pred_te, squared=False))


Derived # of state groups: 50
First 10 group sizes:
 state_group
0    44
1    10
2    44
3    44
4    44
5    44
6    44
7    44
8    44
9    44
Name: count, dtype: int64


INFO     [merf.py:307] Training GLL is 6598.840007552962 at iteration 1.
INFO     [merf.py:307] Training GLL is 6543.059052563239 at iteration 2.
INFO     [merf.py:307] Training GLL is 6504.205413058526 at iteration 3.
INFO     [merf.py:307] Training GLL is 6476.706636652364 at iteration 4.
INFO     [merf.py:307] Training GLL is 6444.936594394965 at iteration 5.
INFO     [merf.py:307] Training GLL is 6430.6331927418105 at iteration 6.
INFO     [merf.py:307] Training GLL is 6421.123105953389 at iteration 7.
INFO     [merf.py:307] Training GLL is 6413.839452454632 at iteration 8.
INFO     [merf.py:307] Training GLL is 6409.208651099883 at iteration 9.
INFO     [merf.py:307] Training GLL is 6406.990171419416 at iteration 10.
INFO     [merf.py:307] Training GLL is 6407.040857963105 at iteration 11.
INFO     [merf.py:307] Training GLL is 6404.581430865687 at iteration 12.
INFO     [merf.py:307] Training GLL is 6402.447439740721 at iteration 13.
INFO     [merf.py:307] Training GLL is 6403.78


FEATURES USED:
['econ_emp_per_1k', 'state_avg_earnings', 'growth_emp_qoq', 'growth_emp_yoy', 'growth_earn_qoq', 'STATE_% Children in Poverty', 'STATE_% Low Birthweight', 'STATE_Food Environment Index', 'STATE_Primary Care Physicians Rate', 'STATE_Social Association Rate', 'STATE_% Smokers', 'STATE_Preventable Hospitalization Rate', 'STATE_Mentally Unhealthy Days']

TRAIN:
  R2  : 0.9320257712997496
  RMSE: 3.690130444741052

TEST:
  R2  : 0.2546166989054003
  RMSE: 11.641365630400133


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:
df = pd.read_csv(r"C:\Users\liuc\Downloads\CH_ECON_V3.csv")

In [4]:
df

,State,Year,quarter,state_total_pop,state_emp_total,state_hires_total,state_avg_earnings,econ_emp_per_1k,econ_hires_per_1k,econ_hire_rate,...,STATE_% With Access to Exercise Opportunities,STATE_Food Environment Index,STATE_Income Ratio,STATE_Mentally Unhealthy Days,STATE_Physically Unhealthy Days,STATE_Preventable Hospitalization Rate,STATE_Primary Care Physicians Rate,STATE_Social Association Rate,STATE_Teen Birth Rate,STATE_Violent Crime Rate
0,Alabama,2014,1,4843737.0,1484428.0,234866.0,3183.492990,306.463377,48.488595,0.158220,...,51.935239,6.932836,0.000000,4.254047,4.289236,0.000000,62.132991,0.000000,48.186843,410.745302
1,Alabama,2014,2,4843737.0,1488384.0,296168.0,3186.477239,307.280102,61.144525,0.198986,...,51.935239,6.932836,0.000000,4.254047,4.289236,0.000000,62.132991,0.000000,48.186843,410.745302
2,Alabama,2014,3,4843737.0,1510987.0,292385.0,3158.744593,311.946540,60.363517,0.193506,...,51.935239,6.932836,0.000000,4.254047,4.289236,0.000000,62.132991,0.000000,48.186843,410.745302
3,Alabama,2014,4,4843737.0,1509475.0,273242.0,3383.957254,311.634385,56.411403,0.181018,...,51.935239,6.932836,0.000000,4.254047,4.289236,0.000000,62.132991,0.000000,48.186843,410.745302
4,Alabama,2015,1,4854803.0,1510048.0,246062.0,3224.297608,311.042075,50.684240,0.162950,...,64.355654,6.694086,5.030240,4.252503,4.285823,69.088952,62.883135,12.483551,46.968524,410.093125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146,Wyoming,2023,4,578239.0,215710.0,45344.0,4717.370878,373.046439,78.417402,0.210208,...,77.638870,7.779140,4.235807,4.019662,2.849604,2324.557970,70.486996,11.973542,24.269712,0.000000
2147,Wyoming,2024,1,580752.0,208388.0,36781.0,4665.328800,358.824421,63.333402,0.176502,...,77.761949,7.781328,4.259097,4.750646,3.437344,2178.144266,70.319097,12.253024,20.177949,0.000000
2148,Wyoming,2024,2,580752.0,208453.0,59642.0,4530.285609,358.936345,102.697881,0.286117,...,77.761949,7.781328,4.259097,4.750646,3.437344,2178.144266,70.319097,12.253024,20.177949,0.000000
2149,Wyoming,2024,3,580752.0,222565.0,51118.0,4574.083715,383.235873,88.020360,0.229677,...,77.761949,7.781328,4.259097,4.750646,3.437344,2178.144266,70.319097,12.253024,20.177949,0.000000


In [3]:
import numpy as np
import pandas as pd
import warnings
import sys
import os

from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from merf.merf import MERF

# Suppress warnings
warnings.filterwarnings('ignore')

# =============================
# CONFIG
# =============================
TARGET = "econ_hires_per_1k"
USE_LOG_TARGET = True  # Transform target to log scale to handle outliers (NV, ND)

# Expanded Candidate List based on your available columns
# We exclude 'hire_rate' as requested, and exclude the target itself.
CANDIDATE_X = [
    # Econ / Structural
    "econ_emp_per_1k",
    "state_avg_earnings",
    "state_total_pop", 
    
    # Growth Metrics
    "growth_emp_qoq",
    "growth_emp_yoy",
    "growth_earn_qoq",
    
    # Health / Social Determinants (Expanded)
    "STATE_% Adults with Obesity",
    "STATE_% Children in Poverty",
    "STATE_% Children in Single-Parent Households",
    "STATE_% Excessive Drinking",
    "STATE_% Fair or Poor Health",
    "STATE_% Low Birthweight",
    "STATE_% Severe Housing Problems",
    "STATE_% Smokers",
    "STATE_% Some College",          # Added: Workforce skill proxy
    "STATE_% Unemployed",            # Added: Critical labor market indicator
    "STATE_% Uninsured",             # Added: Gig economy proxy
    "STATE_Food Environment Index",
    "STATE_Income Ratio",            # Added: Inequality proxy
    "STATE_Mentally Unhealthy Days",
    "STATE_Physically Unhealthy Days",
    "STATE_Preventable Hospitalization Rate",
    "STATE_Primary Care Physicians Rate",
    "STATE_Social Association Rate",
    "STATE_Violent Crime Rate"
]

EXCLUDE_STATES = {"Alaska"} 
TRAIN_END_YEAR = 2019
N_FOLDS = 4
RANDOM_STATE = 42

# TUNING SETTINGS
TUNE_N_ESTIMATORS = 100 
FINAL_N_ESTIMATORS = 500 

# =============================
# HELPERS
# =============================
class HiddenPrints:
    """Context manager to suppress stdout"""
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def metrics_block(name, y_true, y_pred):
    return {
        "split": name,
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": rmse(y_true, y_pred),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }

def normalize_quarter(q):
    q = pd.to_numeric(q, errors="coerce")
    if pd.notna(q.min()) and q.min() == 0:
        q = q + 1
    return q

def make_time_index(df):
    return (df["Year"].astype(int) * 4 + df["quarter"].astype(int)).astype(int)

# =============================
# 1) DATA PREP & FEATURE ENGINEERING
# =============================
df_work = df.copy()
df_work = df_work[~df_work["State"].isin(EXCLUDE_STATES)].copy()

# Basic Cleanup
df_work["Year"] = pd.to_numeric(df_work["Year"], errors="coerce")
df_work["quarter"] = normalize_quarter(df_work["quarter"])
df_work[TARGET] = pd.to_numeric(df_work[TARGET], errors="coerce")

# Filter Candidate X to what actually exists in df
REAL_X = [c for c in CANDIDATE_X if c in df_work.columns]
for c in REAL_X:
    df_work[c] = pd.to_numeric(df_work[c], errors="coerce")

df_work = df_work.dropna(subset=["State", "Year", "quarter", TARGET]).copy()
df_work["Year"] = df_work["Year"].astype(int)
df_work["quarter"] = df_work["quarter"].astype(int)

# --- Feature Engineering ---

# 1. Time Index
df_work["time_index"] = make_time_index(df_work)
df_work["t_trend"] = df_work["time_index"] - df_work["time_index"].min()

# 2. Seasonality (One Hot)
q_dummies = pd.get_dummies(df_work['quarter'], prefix='Q', drop_first=True)
df_work = pd.concat([df_work, q_dummies], axis=1)
seasonal_cols = list(q_dummies.columns)

# 3. Advanced Lags & Rolling Windows
# We sort by State/Time to ensure shift/rolling works
df_work = df_work.sort_values(['State', 'time_index'])

# Features to apply Lag/Rolling to: All REAL_X + Target
cols_to_transform = REAL_X + [TARGET]

feat_cols = []

for c in cols_to_transform:
    # Lag 1 (Immediate past)
    df_work[f"{c}_lag1"] = df_work.groupby("State")[c].shift(1)
    feat_cols.append(f"{c}_lag1")
    
    # Lag 4 (Year over Year reference)
    df_work[f"{c}_lag4"] = df_work.groupby("State")[c].shift(4)
    feat_cols.append(f"{c}_lag4")
    
    # Rolling Mean 4 (Annual Trend Smoothing) - IMPROVEMENT
    # This helps smooth out quarterly noise
    df_work[f"{c}_roll4"] = df_work.groupby("State")[c].transform(lambda x: x.shift(1).rolling(4).mean())
    feat_cols.append(f"{c}_roll4")

# Add Trend and Seasonality to features
feat_cols += ["t_trend"] + seasonal_cols

# Drop NaN caused by lags
df_model = df_work.dropna(subset=feat_cols + [TARGET]).copy()

# Log Transform Target if enabled (Helps with NV/ND outliers)
if USE_LOG_TARGET:
    df_model[f"{TARGET}_log"] = np.log1p(df_model[TARGET])
    target_col_name = f"{TARGET}_log"
else:
    target_col_name = TARGET

# =============================
# 2) TRAIN/TEST SPLIT
# =============================
df_tr = df_model[df_model["Year"] <= TRAIN_END_YEAR].copy()
df_te = df_model[df_model["Year"] > TRAIN_END_YEAR].copy()

# Drop columns that are all-NaN in training
valid_cols = [c for c in feat_cols if df_tr[c].notna().all()]
print(f"Original Feats: {len(feat_cols)}, Valid Feats: {len(valid_cols)}")

X_cols = valid_cols

y_tr = df_tr[target_col_name].values.astype(float)
y_te = df_te[target_col_name].values.astype(float)

# Groups for MERF
cl_tr = df_tr["State"].astype(str)
cl_te = df_te["State"].astype(str)

# Random Effect Design Matrix (Z) - Just a bias term (ones)
Z_tr = np.ones((len(df_tr), 1), dtype=float)
Z_te = np.ones((len(df_te), 1), dtype=float)

X_tr_raw = df_tr[X_cols]
X_te_raw = df_te[X_cols]

# Impute
imp = SimpleImputer(strategy="median")
X_tr = imp.fit_transform(X_tr_raw)
X_te = imp.transform(X_te_raw)

# =============================
# 3) TUNING (QUIET MODE)
# =============================
print("\nStarting Grid Search (Output suppressed)...")

# Reduced complexity grid to prevent overfitting
param_grid = {
    "max_depth": [6, 8, 12],         # Cap depth to force generalization
    "min_samples_leaf": [5, 10],     # Require more samples per leaf
    "max_features": ["sqrt", 0.3],   # Force tree decorrelation
}

# Walk-Forward Splits
times = sorted(df_tr["time_index"].unique())
blocks = np.array_split(times, N_FOLDS + 1)
wf_splits = []
curr_tr = blocks[0]
for i in range(N_FOLDS):
    if i > 0: curr_tr = np.concatenate([curr_tr, blocks[i]])
    curr_va = blocks[i+1]
    wf_splits.append((set(curr_tr), set(curr_va)))

best_rmse = np.inf
best_params = None

# Using HiddenPrints to silence MERF iteration logs
with HiddenPrints():
    for params in ParameterGrid(param_grid):
        scores = []
        for tr_t, va_t in wf_splits:
            # Indices
            idx_tr = df_tr["time_index"].isin(tr_t)
            idx_va = df_tr["time_index"].isin(va_t)
            
            # Slice
            X_f_tr, y_f_tr, Z_f_tr, cl_f_tr = X_tr[idx_tr], y_tr[idx_tr], Z_tr[idx_tr], cl_tr[idx_tr]
            X_f_va, y_f_va, Z_f_va, cl_f_va = X_tr[idx_va], y_tr[idx_va], Z_tr[idx_va], cl_tr[idx_va]
            
            # Fit
            rf = RandomForestRegressor(n_estimators=TUNE_N_ESTIMATORS, n_jobs=-1, random_state=RANDOM_STATE, **params)
            m = MERF(rf, max_iterations=5) # Low iterations for tuning speed
            m.fit(X_f_tr, Z_f_tr, cl_f_tr, y_f_tr)
            
            p_va = m.predict(X_f_va, Z_f_va, cl_f_va)
            scores.append(np.sqrt(mean_squared_error(y_f_va, p_va)))
        
        avg_score = np.mean(scores)
        if avg_score < best_rmse:
            best_rmse = avg_score
            best_params = params

print(f"Best CV RMSE (Transformed): {best_rmse:.4f}")
print(f"Best Params: {best_params}")

# =============================
# 4) FINAL FIT & EVAL
# =============================
print("\nFitting Final Model...")

rf_final = RandomForestRegressor(n_estimators=FINAL_N_ESTIMATORS, n_jobs=-1, random_state=RANDOM_STATE, **best_params)

# Silence the final fit as well
with HiddenPrints():
    merf_final = MERF(rf_final, max_iterations=20)
    merf_final.fit(X_tr, Z_tr, cl_tr, y_tr)

# Predict
pred_tr_trans = merf_final.predict(X_tr, Z_tr, cl_tr)
pred_te_trans = merf_final.predict(X_te, Z_te, cl_te)

# Inverse Transform if Log used
if USE_LOG_TARGET:
    y_tr_orig = np.expm1(y_tr)
    y_te_orig = np.expm1(y_te)
    pred_tr_orig = np.expm1(pred_tr_trans)
    pred_te_orig = np.expm1(pred_te_trans)
else:
    y_tr_orig = y_tr
    y_te_orig = y_te
    pred_tr_orig = pred_tr_trans
    pred_te_orig = pred_te_trans

print("\n--- RESULTS ---")
print(metrics_block("TRAIN", y_tr_orig, pred_tr_orig))
print(metrics_block("TEST", y_te_orig, pred_te_orig))

# Feature Importance (from the fixed effect RF)
imp_df = pd.DataFrame({
    'feature': X_cols,
    'importance': merf_final.trained_fe_model.feature_importances_
}).sort_values('importance', ascending=False).head(10)

print("\nTop 10 Drivers:")
print(imp_df)

Original Feats: 82, Valid Feats: 82

Starting Grid Search (Output suppressed)...


INFO     [merf.py:307] Training GLL is -389.9590165547215 at iteration 1.
INFO     [merf.py:307] Training GLL is -763.8024796282809 at iteration 2.
INFO     [merf.py:307] Training GLL is -1066.4370615738671 at iteration 3.
INFO     [merf.py:307] Training GLL is -1219.6380098424627 at iteration 4.
INFO     [merf.py:307] Training GLL is -1245.887071634718 at iteration 5.
INFO     [merf.py:307] Training GLL is -802.7825568258776 at iteration 1.
INFO     [merf.py:307] Training GLL is -1530.178423907873 at iteration 2.
INFO     [merf.py:307] Training GLL is -1926.851257422507 at iteration 3.
INFO     [merf.py:307] Training GLL is -1968.843972382961 at iteration 4.
INFO     [merf.py:307] Training GLL is -1974.692369936032 at iteration 5.
INFO     [merf.py:307] Training GLL is -1270.1531646334872 at iteration 1.
INFO     [merf.py:307] Training GLL is -2344.6629981083656 at iteration 2.
INFO     [merf.py:307] Training GLL is -2753.674875127493 at iteration 3.
INFO     [merf.py:307] Training GL

Best CV RMSE (Transformed): 0.0497
Best Params: {'max_depth': 8, 'max_features': 0.3, 'min_samples_leaf': 5}

Fitting Final Model...


INFO     [merf.py:307] Training GLL is -2340.865913413404 at iteration 1.
INFO     [merf.py:307] Training GLL is -4432.421052293918 at iteration 2.
INFO     [merf.py:307] Training GLL is -5232.642321370703 at iteration 3.
INFO     [merf.py:307] Training GLL is -5270.968293472973 at iteration 4.
INFO     [merf.py:307] Training GLL is -5260.1736364500985 at iteration 5.
INFO     [merf.py:307] Training GLL is -5256.38253185449 at iteration 6.
INFO     [merf.py:307] Training GLL is -5255.376030464955 at iteration 7.
INFO     [merf.py:307] Training GLL is -5248.855765475695 at iteration 8.
INFO     [merf.py:307] Training GLL is -5246.2706559197395 at iteration 9.
INFO     [merf.py:307] Training GLL is -5235.7351235019505 at iteration 10.
INFO     [merf.py:307] Training GLL is -5227.699393775909 at iteration 11.
INFO     [merf.py:307] Training GLL is -5234.3327533759475 at iteration 12.
INFO     [merf.py:307] Training GLL is -5238.424584501637 at iteration 13.
INFO     [merf.py:307] Training


--- RESULTS ---
{'split': 'TRAIN', 'r2': 0.9786799013797017, 'rmse': 1.891661322788209, 'mae': 1.2850111471856491}
{'split': 'TEST', 'r2': 0.5262158317967627, 'rmse': 9.76405394411975, 'mae': 7.307097495037219}

Top 10 Drivers:
                    feature  importance
76   econ_hires_per_1k_lag4    0.447869
15     growth_earn_qoq_lag1    0.101357
10      growth_emp_qoq_lag4    0.101035
79                      Q_2    0.063772
77  econ_hires_per_1k_roll4    0.041697
75   econ_hires_per_1k_lag1    0.034116
9       growth_emp_qoq_lag1    0.032600
78                  t_trend    0.017890
8     state_total_pop_roll4    0.010428
16     growth_earn_qoq_lag4    0.009857


In [6]:
import numpy as np
import pandas as pd
import warnings
import sys
import os

from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from merf.merf import MERF

warnings.filterwarnings('ignore')

# =============================
# CONFIG
# =============================
TARGET = "econ_hires_per_1k"
USE_LOG_TARGET = True 

# --- CRITICAL CHANGE: STRUCTURAL FEATURE SET ---
# We removed all "Target Lags" and "QoQ Growth". 
# The model must now rely on Health and Economic Fundamentals.

CANDIDATE_X = [
    # 1. Economic Fundamentals (Levels, not growth)
    "state_avg_earnings",           # Wage theory: Lower wages = higher turnover?
    "STATE_% Unemployed",           # Phillips curve: Labor leverage
    "STATE_% Uninsured",            # Gig economy / Precarious work proxy
    "STATE_% Some College",         # Skill level proxy
    "state_total_pop",              # Scale factor

    # 2. Health & Social Determinants (The Focus)
    "STATE_% Adults with Obesity",
    "STATE_% Children in Poverty",
    "STATE_% Children in Single-Parent Households",
    "STATE_% Excessive Drinking",
    "STATE_% Fair or Poor Health",
    "STATE_% Low Birthweight",
    "STATE_% Severe Housing Problems",
    "STATE_% Smokers",
    "STATE_Food Environment Index",
    "STATE_Income Ratio",
    "STATE_Mentally Unhealthy Days",
    "STATE_Physically Unhealthy Days",
    "STATE_Preventable Hospitalization Rate",
    "STATE_Primary Care Physicians Rate",
    "STATE_Social Association Rate",
    "STATE_Violent Crime Rate"
]

# We will add Seasonality (Q1-Q4) and Time Trend (t_trend) manually below.

EXCLUDE_STATES = {"Alaska"} 
TRAIN_END_YEAR = 2019
N_FOLDS = 4
RANDOM_STATE = 42

# TUNING SETTINGS
TUNE_N_ESTIMATORS = 100 
FINAL_N_ESTIMATORS = 500 

# =============================
# HELPERS
# =============================
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def metrics_block(name, y_true, y_pred):
    return {
        "split": name,
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }

def normalize_quarter(q):
    q = pd.to_numeric(q, errors="coerce")
    if pd.notna(q.min()) and q.min() == 0:
        q = q + 1
    return q

def make_time_index(df):
    return (df["Year"].astype(int) * 4 + df["quarter"].astype(int)).astype(int)

# =============================
# 1) DATA PREP
# =============================
df_work = df.copy()
df_work = df_work[~df_work["State"].isin(EXCLUDE_STATES)].copy()

df_work["Year"] = pd.to_numeric(df_work["Year"], errors="coerce")
df_work["quarter"] = normalize_quarter(df_work["quarter"])
df_work[TARGET] = pd.to_numeric(df_work[TARGET], errors="coerce")

REAL_X = [c for c in CANDIDATE_X if c in df_work.columns]
for c in REAL_X:
    df_work[c] = pd.to_numeric(df_work[c], errors="coerce")

df_work = df_work.dropna(subset=["State", "Year", "quarter", TARGET]).copy()
df_work["Year"] = df_work["Year"].astype(int)
df_work["quarter"] = df_work["quarter"].astype(int)

# Time Features
df_work["time_index"] = make_time_index(df_work)
df_work["t_trend"] = df_work["time_index"] - df_work["time_index"].min()

# Seasonality (Crucial for Hires)
q_dummies = pd.get_dummies(df_work['quarter'], prefix='Q', drop_first=True)
df_work = pd.concat([df_work, q_dummies], axis=1)
seasonal_cols = list(q_dummies.columns)

# Combine Features
# Note: We are NO LONGER generating lags for everything. 
# We are using the "Current Status" to predict "Current Hires".
X_cols = REAL_X + ["t_trend"] + seasonal_cols

# We still need to handle NaNs if any (though unlikely without lags)
df_model = df_work.dropna(subset=X_cols + [TARGET]).copy()

# Log Transform Target
if USE_LOG_TARGET:
    df_model[f"{TARGET}_log"] = np.log1p(df_model[TARGET])
    target_col_name = f"{TARGET}_log"
else:
    target_col_name = TARGET

# =============================
# 2) TRAIN/TEST SPLIT
# =============================
df_tr = df_model[df_model["Year"] <= TRAIN_END_YEAR].copy()
df_te = df_model[df_model["Year"] > TRAIN_END_YEAR].copy()

y_tr = df_tr[target_col_name].values.astype(float)
y_te = df_te[target_col_name].values.astype(float)

cl_tr = df_tr["State"].astype(str)
cl_te = df_te["State"].astype(str)

Z_tr = np.ones((len(df_tr), 1), dtype=float)
Z_te = np.ones((len(df_te), 1), dtype=float)

X_tr_raw = df_tr[X_cols]
X_te_raw = df_te[X_cols]

imp = SimpleImputer(strategy="median")
X_tr = imp.fit_transform(X_tr_raw)
X_te = imp.transform(X_te_raw)

print(f"Training on {len(df_tr)} samples, Testing on {len(df_te)} samples.")
print(f"Feature Count: {len(X_cols)}")

# =============================
# 3) TUNING
# =============================
print("\nStarting Grid Search...")
# Increased depth slightly because we removed the strong lag features. 
# The model needs more complexity to find the subtle health patterns.
param_grid = {
    "max_depth": [8, 12, 15],         
    "min_samples_leaf": [5, 10],     
    "max_features": ["sqrt", 0.4],   
}

times = sorted(df_tr["time_index"].unique())
blocks = np.array_split(times, N_FOLDS + 1)
wf_splits = []
curr_tr = blocks[0]
for i in range(N_FOLDS):
    if i > 0: curr_tr = np.concatenate([curr_tr, blocks[i]])
    curr_va = blocks[i+1]
    wf_splits.append((set(curr_tr), set(curr_va)))

best_rmse = np.inf
best_params = None

with HiddenPrints():
    for params in ParameterGrid(param_grid):
        scores = []
        for tr_t, va_t in wf_splits:
            idx_tr = df_tr["time_index"].isin(tr_t)
            idx_va = df_tr["time_index"].isin(va_t)
            
            # Skip if split is empty
            if not any(idx_tr) or not any(idx_va): continue

            X_f_tr, y_f_tr, Z_f_tr, cl_f_tr = X_tr[idx_tr], y_tr[idx_tr], Z_tr[idx_tr], cl_tr[idx_tr]
            X_f_va, y_f_va, Z_f_va, cl_f_va = X_tr[idx_va], y_tr[idx_va], Z_tr[idx_va], cl_tr[idx_va]
            
            rf = RandomForestRegressor(n_estimators=TUNE_N_ESTIMATORS, n_jobs=-1, random_state=RANDOM_STATE, **params)
            m = MERF(rf, max_iterations=5)
            m.fit(X_f_tr, Z_f_tr, cl_f_tr, y_f_tr)
            
            p_va = m.predict(X_f_va, Z_f_va, cl_f_va)
            scores.append(np.sqrt(mean_squared_error(y_f_va, p_va)))
        
        if scores:
            avg_score = np.mean(scores)
            if avg_score < best_rmse:
                best_rmse = avg_score
                best_params = params

print(f"Best CV RMSE: {best_rmse:.4f}")
print(f"Best Params: {best_params}")

# =============================
# 4) FINAL FIT
# =============================
print("\nFitting Final Model...")
rf_final = RandomForestRegressor(n_estimators=FINAL_N_ESTIMATORS, n_jobs=-1, random_state=RANDOM_STATE, **best_params)

with HiddenPrints():
    merf_final = MERF(rf_final, max_iterations=20)
    merf_final.fit(X_tr, Z_tr, cl_tr, y_tr)

pred_tr_trans = merf_final.predict(X_tr, Z_tr, cl_tr)
pred_te_trans = merf_final.predict(X_te, Z_te, cl_te)

if USE_LOG_TARGET:
    y_tr_orig = np.expm1(y_tr)
    y_te_orig = np.expm1(y_te)
    pred_tr_orig = np.expm1(pred_tr_trans)
    pred_te_orig = np.expm1(pred_te_trans)
else:
    y_tr_orig = y_tr
    y_te_orig = y_te
    pred_tr_orig = pred_tr_trans
    pred_te_orig = pred_te_trans

print("\n--- RESULTS ---")
print(metrics_block("TRAIN", y_tr_orig, pred_tr_orig))
print(metrics_block("TEST", y_te_orig, pred_te_orig))

imp_df = pd.DataFrame({
    'feature': X_cols,
    'importance': merf_final.trained_fe_model.feature_importances_
}).sort_values('importance', ascending=False).head(12)

print("\nTop 12 Drivers (Structural & Health):")
print(imp_df)

INFO     [merf.py:307] Training GLL is -515.5065244241347 at iteration 1.


Training on 1176 samples, Testing on 965 samples.
Feature Count: 25

Starting Grid Search...


INFO     [merf.py:307] Training GLL is -981.5883954604631 at iteration 2.
INFO     [merf.py:307] Training GLL is -1270.0948334693417 at iteration 3.
INFO     [merf.py:307] Training GLL is -1360.6272367426488 at iteration 4.
INFO     [merf.py:307] Training GLL is -1381.6622625057528 at iteration 5.
INFO     [merf.py:307] Training GLL is -1235.1368924525468 at iteration 1.
INFO     [merf.py:307] Training GLL is -2145.8203813757277 at iteration 2.
INFO     [merf.py:307] Training GLL is -2340.9001230339163 at iteration 3.
INFO     [merf.py:307] Training GLL is -2352.5856106325587 at iteration 4.
INFO     [merf.py:307] Training GLL is -2364.7000630776647 at iteration 5.
INFO     [merf.py:307] Training GLL is -2054.2275645098343 at iteration 1.
INFO     [merf.py:307] Training GLL is -3273.5287210966317 at iteration 2.
INFO     [merf.py:307] Training GLL is -3499.511461248862 at iteration 3.
INFO     [merf.py:307] Training GLL is -3539.5035127421893 at iteration 4.
INFO     [merf.py:307] Trai

Best CV RMSE: 0.0976
Best Params: {'max_depth': 15, 'max_features': 0.4, 'min_samples_leaf': 5}

Fitting Final Model...


INFO     [merf.py:307] Training GLL is -3870.8675490942383 at iteration 1.
INFO     [merf.py:307] Training GLL is -6746.383607277591 at iteration 2.
INFO     [merf.py:307] Training GLL is -7197.555411067376 at iteration 3.
INFO     [merf.py:307] Training GLL is -7244.178007972446 at iteration 4.
INFO     [merf.py:307] Training GLL is -7278.204437394342 at iteration 5.
INFO     [merf.py:307] Training GLL is -7309.340217718938 at iteration 6.
INFO     [merf.py:307] Training GLL is -7317.451593306581 at iteration 7.
INFO     [merf.py:307] Training GLL is -7317.884391084632 at iteration 8.
INFO     [merf.py:307] Training GLL is -7323.84853448706 at iteration 9.
INFO     [merf.py:307] Training GLL is -7330.384558296573 at iteration 10.
INFO     [merf.py:307] Training GLL is -7346.358340526577 at iteration 11.
INFO     [merf.py:307] Training GLL is -7355.729110285766 at iteration 12.
INFO     [merf.py:307] Training GLL is -7362.199863056326 at iteration 13.
INFO     [merf.py:307] Training GL


--- RESULTS ---
{'split': 'TRAIN', 'r2': 0.9706942927979709, 'rmse': 2.3563835747318023, 'mae': 1.5091733442876165}
{'split': 'TEST', 'r2': 0.49087541076370655, 'rmse': 10.121664418518609, 'mae': 7.6325814881300245}

Top 12 Drivers (Structural & Health):
                          feature  importance
22                            Q_2    0.271545
23                            Q_3    0.191395
24                            Q_4    0.139632
21                        t_trend    0.130629
4                 state_total_pop    0.032168
3            STATE_% Some College    0.023925
2               STATE_% Uninsured    0.023090
1              STATE_% Unemployed    0.019408
0              state_avg_earnings    0.017856
6     STATE_% Children in Poverty    0.017375
9     STATE_% Fair or Poor Health    0.015828
19  STATE_Social Association Rate    0.013839


In [33]:
df1 = pd.read_csv(r"C:\Users\liuc\Desktop\CH_ECON_V4.csv")

In [35]:
df1.columns

Index(['Unnamed: 0', 'State', 'Year', 'state_total_pop', 'state_emp_avg',
       'state_emp_sum', 'state_hires_total', 'state_avg_earnings',
       'state_avg_earnings_meanq', 'econ_emp_per_1k', 'econ_hires_per_1k',
       'econ_hire_rate_annual', 'growth_emp_yoy', 'growth_earn_yoy',
       'growth_hires_yoy', 'STATE_# Alcohol-Impaired Driving Deaths',
       'STATE_# Driving Deaths', 'STATE_Premature Deaths',
       'STATE_% Adults with Obesity', 'STATE_% Children in Poverty',
       'STATE_% Children in Single-Parent Households',
       'STATE_% Drive Alone to Work', 'STATE_% Excessive Drinking',
       'STATE_% Fair or Poor Health', 'STATE_% Long Commute - Drives Alone',
       'STATE_% Low Birthweight', 'STATE_% Severe Housing Problems',
       'STATE_% Smokers', 'STATE_% Some College', 'STATE_% Unemployed',
       'STATE_% Uninsured', 'STATE_% With Access to Exercise Opportunities',
       'STATE_Food Environment Index', 'STATE_Income Ratio',
       'STATE_Mentally Unhealthy Days'

In [32]:
import numpy as np
import pandas as pd
import warnings
import sys
import os

from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor

from merf.merf import MERF

warnings.filterwarnings("ignore")


# ============================================================
# CONFIG
# ============================================================
TARGET = "econ_hires_per_1k"
USE_LOG_TARGET = True

START_YEAR = 2014
END_YEAR   = 2024
TRAIN_END_YEAR = 2019

EXCLUDE_STATES = {"Alaska"}   # optional
RANDOM_STATE = 42

# MERF / RF tuning and final fit
TUNE_N_ESTIMATORS  = 150
FINAL_N_ESTIMATORS = 700
MERF_TUNE_ITERS    = 5
MERF_FINAL_ITERS   = 20

# Walk-forward tuning folds within train window (<=2019)
MIN_TRAIN_YEARS_TUNE = 3

# Optional macro feature (requires state totals and population columns)
ADD_US_MACRO = True
STATE_HIRES_TOTAL_COL = "state_hires_total"
STATE_POP_COL         = "state_total_pop"

# Candidate feature set (annual, levels)
CANDIDATE_X = [
    # Economic fundamentals (levels)
    "state_avg_earnings",
    "STATE_% Unemployed",
    "STATE_% Uninsured",
    "STATE_% Some College",
    "state_total_pop",

    # Health & social determinants
    "STATE_% Adults with Obesity",
    "STATE_% Children in Poverty",
    "STATE_% Children in Single-Parent Households",
    "STATE_% Excessive Drinking",
    "STATE_% Fair or Poor Health",
    "STATE_% Low Birthweight",
    "STATE_% Severe Housing Problems",
    "STATE_% Smokers",
    "STATE_Food Environment Index",
    "STATE_Income Ratio",
    "STATE_Mentally Unhealthy Days",
    "STATE_Physically Unhealthy Days",
    "STATE_Preventable Hospitalization Rate",
    "STATE_Primary Care Physicians Rate",
    "STATE_Social Association Rate",
    "STATE_Violent Crime Rate",
]

# Leakage columns (drop if present)
LEAKAGE_COLS = [
    TARGET,
    "econ_hire_rate_annual",
    "growth_hires_yoy",
    STATE_HIRES_TOTAL_COL,
]


# ============================================================
# HELPERS
# ============================================================
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def metrics_block(name, y_true, y_pred):
    return {
        "split": name,
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": rmse(y_true, y_pred),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "n": int(len(y_true)),
    }

def make_walk_forward_splits_by_year(years: np.ndarray, min_train_years: int = 3):
    years = np.asarray(years).astype(int)
    uniq = np.sort(np.unique(years))
    if len(uniq) < min_train_years + 1:
        raise ValueError("Not enough unique years for walk-forward folds.")
    splits = []
    for i in range(min_train_years, len(uniq)):
        tr_years = uniq[:i]
        va_year  = uniq[i]
        tr_idx = np.where(np.isin(years, tr_years))[0]
        va_idx = np.where(years == va_year)[0]
        if len(tr_idx) and len(va_idx):
            splits.append((tr_idx, va_idx))
    if not splits:
        raise ValueError("No walk-forward splits created.")
    return splits

def safe_intersection(cols, df_cols):
    return [c for c in cols if c in df_cols]

def backtransform_if_needed(y, use_log):
    return np.expm1(y) if use_log else y


# ============================================================
# 1) LOAD / CLEAN ANNUAL STATE-YEAR PANEL (df1 must exist)
# ============================================================
df0 = df1.copy()

for c in ["Unnamed: 0", "Unnamed: 0.1", "index"]:
    if c in df0.columns:
        df0 = df0.drop(columns=[c])

if "State" not in df0.columns or "Year" not in df0.columns:
    raise ValueError("df1 must contain at least columns: ['State','Year'].")

df0["State"] = df0["State"].astype(str)
df0["Year"]  = pd.to_numeric(df0["Year"], errors="coerce")
df0 = df0.dropna(subset=["State", "Year"]).copy()
df0["Year"]  = df0["Year"].astype(int)

df0 = df0[(df0["Year"] >= START_YEAR) & (df0["Year"] <= END_YEAR)].copy()

if EXCLUDE_STATES:
    df0 = df0[~df0["State"].isin(EXCLUDE_STATES)].copy()

if TARGET not in df0.columns:
    raise ValueError(f"Target '{TARGET}' not found in df1 columns.")

df0[TARGET] = pd.to_numeric(df0[TARGET], errors="coerce")
df0 = df0.dropna(subset=[TARGET]).copy()

# Optional US macro feature
if ADD_US_MACRO and (STATE_HIRES_TOTAL_COL in df0.columns) and (STATE_POP_COL in df0.columns):
    df0[STATE_HIRES_TOTAL_COL] = pd.to_numeric(df0[STATE_HIRES_TOTAL_COL], errors="coerce")
    df0[STATE_POP_COL] = pd.to_numeric(df0[STATE_POP_COL], errors="coerce")
    us_macro = (
        df0.groupby("Year", as_index=False)
           .agg(US_hires_sum=(STATE_HIRES_TOTAL_COL, "sum"),
                US_pop_sum=(STATE_POP_COL, "sum"))
    )
    us_macro["US_hires_per_1k"] = 1000.0 * us_macro["US_hires_sum"] / us_macro["US_pop_sum"].replace({0: np.nan})
    df0 = df0.merge(us_macro[["Year", "US_hires_per_1k"]], on="Year", how="left")
else:
    ADD_US_MACRO = False

# Time / structural break features
df0["t_trend"] = df0["Year"] - df0["Year"].min()
df0["post_2020"] = (df0["Year"] >= 2020).astype(int)
df0["t_post2020"] = (df0["Year"] - 2020).clip(lower=0)

if "state_total_pop" in df0.columns:
    df0["state_total_pop"] = pd.to_numeric(df0["state_total_pop"], errors="coerce")
    df0["log_pop"] = np.log(df0["state_total_pop"].replace({0: np.nan}))

REAL_X = safe_intersection(CANDIDATE_X, df0.columns)
REAL_X = [c for c in REAL_X if c not in set(LEAKAGE_COLS)]  # avoid accidental leakage

extra_feats = ["t_trend", "post_2020", "t_post2020"]
if "log_pop" in df0.columns and "log_pop" not in REAL_X:
    extra_feats.append("log_pop")
if ADD_US_MACRO:
    extra_feats.append("US_hires_per_1k")

X_cols = [c for c in (REAL_X + extra_feats) if c in df0.columns]
if len(X_cols) == 0:
    raise ValueError("No usable features found. Check CANDIDATE_X against df1 columns.")

df_model = df0.dropna(subset=["State", "Year", TARGET]).copy()

if USE_LOG_TARGET:
    df_model[f"{TARGET}_log"] = np.log1p(df_model[TARGET].astype(float))
    target_col = f"{TARGET}_log"
else:
    target_col = TARGET

print("Annual panel rows:", df_model.shape[0])
print("Years:", sorted(df_model["Year"].unique()))
print("Feature count:", len(X_cols))
print("Using US macro feature:", ADD_US_MACRO)


# ============================================================
# 2) TRAIN/TEST SPLIT
# ============================================================
df_tr = df_model[df_model["Year"] <= TRAIN_END_YEAR].copy()
df_te = df_model[df_model["Year"] >  TRAIN_END_YEAR].copy()

# MERF needs test clusters to exist in train
train_states = set(df_tr["State"].unique())
df_te = df_te[df_te["State"].isin(train_states)].copy()

y_tr = df_tr[target_col].to_numpy(dtype=float)
y_te = df_te[target_col].to_numpy(dtype=float)

# IMPORTANT: MERF requires clusters to be a pandas Series
cl_tr = df_tr["State"].astype(str)   # keep as Series (NOT .values)
cl_te = df_te["State"].astype(str)

Z_tr = np.ones((len(df_tr), 1), dtype=float)
Z_te = np.ones((len(df_te), 1), dtype=float)

X_tr_raw = df_tr[X_cols].copy()
X_te_raw = df_te[X_cols].copy()

years_tr = df_tr["Year"].astype(int).to_numpy()
wf_splits = make_walk_forward_splits_by_year(years_tr, min_train_years=MIN_TRAIN_YEARS_TUNE)

print(f"Train n={len(df_tr)} | Test n={len(df_te)}")
print("Train years:", sorted(df_tr["Year"].unique()))
print("Test years :", sorted(df_te["Year"].unique()))
print("Walk-forward tuning folds:", len(wf_splits))


# ============================================================
# 3) TUNING (WALK-FORWARD CV within train)
# ============================================================
print("\nStarting MERF grid search (annual, walk-forward by year)...")

param_grid = {
    "max_depth": [4, 6, 8, 10],
    "min_samples_leaf": [10, 20, 30],
    "min_samples_split": [20, 40, 60],
    "max_features": [0.4, 0.6, "sqrt"],
}

best_rmse = np.inf
best_params = None

with HiddenPrints():
    for params in ParameterGrid(param_grid):
        fold_scores = []

        for tr_idx, va_idx in wf_splits:
            X_f_tr_raw = X_tr_raw.iloc[tr_idx]
            y_f_tr     = y_tr[tr_idx]
            Z_f_tr     = Z_tr[tr_idx]
            cl_f_tr    = cl_tr.iloc[tr_idx]   # Series

            X_f_va_raw = X_tr_raw.iloc[va_idx]
            y_f_va     = y_tr[va_idx]
            Z_f_va     = Z_tr[va_idx]
            cl_f_va    = cl_tr.iloc[va_idx]   # Series

            imp = SimpleImputer(strategy="median")
            X_f_tr = imp.fit_transform(X_f_tr_raw)
            X_f_va = imp.transform(X_f_va_raw)

            rf = RandomForestRegressor(
                n_estimators=TUNE_N_ESTIMATORS,
                n_jobs=-1,
                random_state=RANDOM_STATE,
                **params
            )
            m = MERF(rf, max_iterations=MERF_TUNE_ITERS)
            m.fit(X_f_tr, Z_f_tr, cl_f_tr, y_f_tr)

            p_va = m.predict(X_f_va, Z_f_va, cl_f_va)
            fold_scores.append(rmse(y_f_va, p_va))

        if fold_scores:
            avg_score = float(np.mean(fold_scores))
            if avg_score < best_rmse:
                best_rmse = avg_score
                best_params = params

print(f"Best CV RMSE (model scale): {best_rmse:.5f}")
print(f"Best RF params: {best_params}")


# ============================================================
# 4) FINAL FIT + EVAL
# ============================================================
print("\nFitting final MERF model...")

imp_final = SimpleImputer(strategy="median")
X_tr = imp_final.fit_transform(X_tr_raw)
X_te = imp_final.transform(X_te_raw)

rf_final = RandomForestRegressor(
    n_estimators=FINAL_N_ESTIMATORS,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    **(best_params if best_params is not None else {})
)

with HiddenPrints():
    merf_final = MERF(rf_final, max_iterations=MERF_FINAL_ITERS)
    merf_final.fit(X_tr, Z_tr, cl_tr, y_tr)

pred_tr = merf_final.predict(X_tr, Z_tr, cl_tr)
pred_te = merf_final.predict(X_te, Z_te, cl_te)

y_tr_orig    = backtransform_if_needed(y_tr, USE_LOG_TARGET)
y_te_orig    = backtransform_if_needed(y_te, USE_LOG_TARGET)
pred_tr_orig = backtransform_if_needed(pred_tr, USE_LOG_TARGET)
pred_te_orig = backtransform_if_needed(pred_te, USE_LOG_TARGET)

print("\n--- RESULTS (original scale) ---")
print(metrics_block("TRAIN", y_tr_orig, pred_tr_orig))
print(metrics_block("TEST",  y_te_orig, pred_te_orig))

# Feature importances (fixed effects RF)
try:
    imp_df = pd.DataFrame({
        "feature": X_cols,
        "importance": merf_final.trained_fe_model.feature_importances_
    }).sort_values("importance", ascending=False)

    print("\nTop 15 fixed-effect drivers:")
    print(imp_df.head(15).to_string(index=False))
except Exception as e:
    print("\nFeature importances not available:", repr(e))


# ============================================================
# 5) OPTIONAL: Rolling-origin backtest (2020..2024)
# ============================================================
DO_ROLLING_ORIGIN = True
ROLLING_WINDOW_YEARS = None   # e.g. 8 for rolling, None for expanding
RETUNE_EACH_YEAR = False      # True = slow

def rolling_origin_backtest(df_model, start_test_year=2020, end_test_year=2024,
                            window_years=None, retune_each_year=False):
    rows = []

    for test_year in range(start_test_year, end_test_year + 1):
        train_year_max = test_year - 1

        if window_years is None:
            df_tr_y = df_model[df_model["Year"] <= train_year_max].copy()
        else:
            df_tr_y = df_model[(df_model["Year"] <= train_year_max) &
                               (df_model["Year"] >= train_year_max - window_years + 1)].copy()

        df_te_y = df_model[df_model["Year"] == test_year].copy()

        # Keep only states seen in train
        seen = set(df_tr_y["State"].unique())
        df_te_y = df_te_y[df_te_y["State"].isin(seen)].copy()

        if df_tr_y.empty or df_te_y.empty:
            continue

        y_tr_y = df_tr_y[target_col].to_numpy(dtype=float)
        y_te_y = df_te_y[target_col].to_numpy(dtype=float)

        cl_tr_y = df_tr_y["State"].astype(str)  # Series
        cl_te_y = df_te_y["State"].astype(str)  # Series

        Z_tr_y = np.ones((len(df_tr_y), 1), dtype=float)
        Z_te_y = np.ones((len(df_te_y), 1), dtype=float)

        X_tr_y_raw = df_tr_y[X_cols].copy()
        X_te_y_raw = df_te_y[X_cols].copy()

        params_use = best_params

        if retune_each_year:
            years_tmp = df_tr_y["Year"].astype(int).to_numpy()
            uniq_years = np.unique(years_tmp)
            if len(uniq_years) >= 4:
                splits_tmp = make_walk_forward_splits_by_year(years_tmp, min_train_years=3)
                best_rmse_tmp = np.inf
                best_params_tmp = None

                for params in ParameterGrid(param_grid):
                    fold_scores = []
                    for tr_idx, va_idx in splits_tmp:
                        imp = SimpleImputer(strategy="median")
                        X_f_tr = imp.fit_transform(X_tr_y_raw.iloc[tr_idx])
                        X_f_va = imp.transform(X_tr_y_raw.iloc[va_idx])

                        rf = RandomForestRegressor(
                            n_estimators=TUNE_N_ESTIMATORS,
                            n_jobs=-1,
                            random_state=RANDOM_STATE,
                            **params
                        )
                        m = MERF(rf, max_iterations=MERF_TUNE_ITERS)
                        with HiddenPrints():
                            m.fit(X_f_tr, Z_tr_y[tr_idx], cl_tr_y.iloc[tr_idx], y_tr_y[tr_idx])
                        p_va = m.predict(X_f_va, Z_tr_y[va_idx], cl_tr_y.iloc[va_idx])
                        fold_scores.append(rmse(y_tr_y[va_idx], p_va))
                    if fold_scores:
                        avg = float(np.mean(fold_scores))
                        if avg < best_rmse_tmp:
                            best_rmse_tmp = avg
                            best_params_tmp = params

                params_use = best_params_tmp if best_params_tmp is not None else best_params

        imp = SimpleImputer(strategy="median")
        X_tr_y = imp.fit_transform(X_tr_y_raw)
        X_te_y = imp.transform(X_te_y_raw)

        rf = RandomForestRegressor(
            n_estimators=FINAL_N_ESTIMATORS,
            n_jobs=-1,
            random_state=RANDOM_STATE,
            **(params_use if params_use is not None else {})
        )
        m = MERF(rf, max_iterations=MERF_FINAL_ITERS)
        with HiddenPrints():
            m.fit(X_tr_y, Z_tr_y, cl_tr_y, y_tr_y)
        p_te_y = m.predict(X_te_y, Z_te_y, cl_te_y)

        y_true = backtransform_if_needed(y_te_y, USE_LOG_TARGET)
        y_pred = backtransform_if_needed(p_te_y, USE_LOG_TARGET)

        rows.append({
            "test_year": test_year,
            **metrics_block(f"YEAR_{test_year}", y_true, y_pred)
        })

    return pd.DataFrame(rows)

if DO_ROLLING_ORIGIN:
    bt = rolling_origin_backtest(
        df_model=df_model,
        start_test_year=2020,
        end_test_year=2024,
        window_years=ROLLING_WINDOW_YEARS,
        retune_each_year=RETUNE_EACH_YEAR
    )
    print("\n--- ROLLING-ORIGIN BACKTEST (original scale) ---")
    if bt.empty:
        print("No backtest rows produced (check data coverage / states).")
    else:
        print(bt[["test_year", "r2", "rmse", "mae", "n"]].to_string(index=False))


Annual panel rows: 536
Years: [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Feature count: 26
Using US macro feature: True
Train n=294 | Test n=242
Train years: [2014, 2015, 2016, 2017, 2018, 2019]
Test years : [2020, 2021, 2022, 2023, 2024]
Walk-forward tuning folds: 3

Starting MERF grid search (annual, walk-forward by year)...
Best CV RMSE (model scale): 0.03365
Best RF params: {'max_depth': 10, 'max_features': 0.6, 'min_samples_leaf': 10, 'min_samples_split': 20}

Fitting final MERF model...

--- RESULTS (original scale) ---
{'split': 'TRAIN', 'r2': 0.9677903507348995, 'rmse': 7.556997226316377, 'mae': 4.1805998050578745, 'n': 294}
{'split': 'TEST', 'r2': 0.569871640779752, 'rmse': 29.431456795827298, 'mae': 22.731281696586546, 'n': 242}

Top 15 fixed-effect drivers:
                                     feature  importance
                                     log_pop    0.253215
                             state_total_pop    0.246403
                          

In [23]:
# ============================================================
# STATE-YEAR HIRING MODEL (NON-MERF)
#  - Target: econ_hires_per_1k (annual hires per 1k population)
#  - Walk-forward CV within train (<=2019)
#  - Baselines: LASSO / XGB / MLP
#  - Regularized/Tuned: LASSO / tuned XGB / regularized RF
#  - Stacking: OOF walk-forward Ridge meta-learner (leakage-safe)
#  - Optional rolling-origin check
# ============================================================

import numpy as np
import pandas as pd

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LassoCV, Lasso, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import xgboost as xgb


# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
def root_mean_squared_error(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def eval_metrics(y_true, y_pred):
    return {
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": root_mean_squared_error(y_true, y_pred),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }

def print_perf(title, y_train, pred_train, y_test, pred_test, train_mask=None):
    """
    train_mask: optional boolean mask for selecting valid rows (e.g., for OOF-stacked train preds).
    """
    if train_mask is None:
        train_mask = np.ones(len(y_train), dtype=bool)

    tr = eval_metrics(y_train[train_mask], pred_train[train_mask])
    te = eval_metrics(y_test, pred_test)

    print(f"\n{title}")
    print(f"  Train: R2={tr['r2']:.3f}, RMSE={tr['rmse']:.3f}, MAE={tr['mae']:.3f}   (n={int(train_mask.sum())})")
    print(f"  Test : R2={te['r2']:.3f}, RMSE={te['rmse']:.3f}, MAE={te['mae']:.3f}   (n={len(y_test)})")

    return tr, te

def make_walk_forward_folds(years: np.ndarray, min_train_years: int = 1):
    """
    Walk-forward folds by YEAR.

    For years 2014..2019:
      min_train_years=1 -> 2014->2015, 2014-15->2016, ..., 2014-18->2019
      min_train_years=3 -> 2014-16->2017, 2014-17->2018, 2014-18->2019
    """
    years = np.asarray(years).astype(int)
    uniq = np.sort(np.unique(years))
    if len(uniq) < (min_train_years + 1):
        raise ValueError("Not enough unique years for walk-forward folds.")

    folds = []
    for i in range(min_train_years, len(uniq)):
        tr_years = uniq[:i]
        va_year = uniq[i]
        tr_idx = np.where(np.isin(years, tr_years))[0]
        va_idx = np.where(years == va_year)[0]
        if len(tr_idx) and len(va_idx):
            folds.append((tr_idx, va_idx))

    if not folds:
        raise ValueError("No folds created.")
    return folds

def oof_preds_walk_forward(model_fixed, X_train, y_train, folds):
    """
    Returns OOF predictions length len(y_train), NaN for rows never in validation (usually first year).
    model_fixed must NOT include internal CV with global folds.
    """
    y_train = np.asarray(y_train)
    oof = np.full(len(y_train), np.nan, dtype=float)

    for tr_idx, va_idx in folds:
        m = clone(model_fixed)
        m.fit(X_train.iloc[tr_idx], y_train[tr_idx])
        oof[va_idx] = m.predict(X_train.iloc[va_idx])

    return oof

def stack_with_oof_walk_forward(base_models_fixed, X_train, y_train, X_test, folds_oof, meta_alphas=None):
    """
    Stacking:
      - meta-train uses walk-forward OOF predictions (true out-of-sample)
      - meta-learner fit only on rows where all OOF preds exist (drops earliest year)
      - meta-test uses base predictions trained on full training
    """
    if meta_alphas is None:
        meta_alphas = np.logspace(-3, 3, 13)

    y_train = np.asarray(y_train)

    meta_train = np.column_stack([
        oof_preds_walk_forward(m, X_train, y_train, folds_oof)
        for m in base_models_fixed.values()
    ])

    valid = np.all(np.isfinite(meta_train), axis=1)
    dropped = int((~valid).sum())
    kept = int(valid.sum())

    if kept < 30:
        raise RuntimeError(f"Too few valid OOF rows for meta-learner: kept={kept}, dropped={dropped}")

    # Fit base models on full training for meta-test
    meta_test_cols = []
    for m in base_models_fixed.values():
        mf = clone(m)
        mf.fit(X_train, y_train)
        meta_test_cols.append(mf.predict(X_test))
    meta_test = np.column_stack(meta_test_cols)

    # Meta-learner (RidgeCV)
    meta = RidgeCV(alphas=meta_alphas, cv=5)
    meta.fit(meta_train[valid], y_train[valid])

    ens_train_pred = np.full(len(y_train), np.nan, dtype=float)
    ens_train_pred[valid] = meta.predict(meta_train[valid])
    ens_test_pred = meta.predict(meta_test)

    weights = pd.Series(meta.coef_, index=[f"pred_{k}" for k in base_models_fixed.keys()])
    info = {"meta_train_kept": kept, "meta_train_dropped": dropped, "meta_alpha": float(meta.alpha_)}
    return ens_train_pred, ens_test_pred, valid, weights, info

def fit_eval_models(models, X_train, y_train, X_test, y_test):
    rows = []
    fitted = {}
    for name, model in models.items():
        m = clone(model)
        m.fit(X_train, y_train)
        fitted[name] = m

        pred_tr = m.predict(X_train)
        pred_te = m.predict(X_test)

        tr = eval_metrics(y_train, pred_tr)
        te = eval_metrics(y_test, pred_te)

        rows.append({
            "model": name,
            "r2_train": tr["r2"], "rmse_train": tr["rmse"], "mae_train": tr["mae"],
            "r2_test": te["r2"], "rmse_test": te["rmse"], "mae_test": te["mae"],
        })
    return pd.DataFrame(rows).sort_values("r2_test", ascending=False), fitted

def rolling_origin_eval(model, df_model, X_all, y_all, start_train_end=2017, end_year=2024):
    years = df_model["Year"].astype(int).values
    out = []
    for test_year in range(start_train_end + 1, end_year + 1):
        tr_mask = years <= (test_year - 1)
        te_mask = years == test_year
        if tr_mask.sum() < 30 or te_mask.sum() < 10:
            continue
        m = clone(model)
        m.fit(X_all.loc[tr_mask], y_all[tr_mask])
        pred = m.predict(X_all.loc[te_mask])
        met = eval_metrics(y_all[te_mask], pred)
        out.append({"test_year": test_year, "train_n": int(tr_mask.sum()), "test_n": int(te_mask.sum()), **met})
    return pd.DataFrame(out)


# ============================================================
# 0) LOAD / CLEAN DATA (df1 already loaded by you)
# ============================================================
df0 = df1.copy()

# drop index-like columns
for c in ["Unnamed: 0", "Unnamed: 0.1"]:
    if c in df0.columns:
        df0 = df0.drop(columns=[c])

df0["State"] = df0["State"].astype(str)
df0["Year"] = pd.to_numeric(df0["Year"], errors="coerce")
df0 = df0.dropna(subset=["State", "Year"]).copy()
df0["Year"] = df0["Year"].astype(int)

# Year window
df0 = df0[(df0["Year"] >= 2014) & (df0["Year"] <= 2024)].copy()

# Target (annual hires per 1k)
target = "econ_hires_per_1k"
if target not in df0.columns:
    raise ValueError(f"Target '{target}' not found in df1 columns.")

df0[target] = pd.to_numeric(df0[target], errors="coerce")
df_model = df0[np.isfinite(df0[target])].copy()

# Optional: log target for heavy tails/outliers
USE_LOG_TARGET = True
y_raw = df_model[target].values.astype(float)
y = np.log1p(y_raw) if USE_LOG_TARGET else y_raw


# ============================================================
# 1) BUILD LEAKAGE-SAFE FEATURE MATRIX X
# ============================================================
id_cols = ["State", "Year"]

# Anything that directly contains hires or is computed from hires should be excluded
# (otherwise the model "cheats" and looks great in train)
hire_leakage = [
    "state_hires_total",        # target numerator in most constructions
    "econ_hire_rate_annual",    # uses hires/employment
    "growth_hires_yoy",         # computed from hires series
]

# Also exclude the raw target itself
exclude_cols = set(id_cols + hire_leakage + [target])

X = df_model.drop(columns=[c for c in exclude_cols if c in df_model.columns], errors="ignore")

# numeric only
X = X.select_dtypes(include=[np.number]).copy()
X = X.dropna(axis=1, how="all")
X = X.loc[:, X.nunique(dropna=True) > 1]

# Small stabilizers that often help annual models:
# (if these exist already, skip)
if "t_trend" not in X.columns:
    X["t_trend"] = df_model["Year"].astype(int) - df_model["Year"].astype(int).min()
if "log_pop" not in X.columns and "state_total_pop" in df_model.columns:
    X["log_pop"] = np.log(df_model["state_total_pop"].replace({0: np.nan}))

print("Final feature count:", X.shape[1])
print("Total rows for modeling:", X.shape[0])


# ============================================================
# 2) TIME SPLIT + FOLDS
# ============================================================
split_year = 2019
train_mask = df_model["Year"] <= split_year
test_mask  = df_model["Year"] >  split_year

X_train, X_test = X.loc[train_mask].copy(), X.loc[test_mask].copy()
y_train, y_test = y[train_mask.values], y[test_mask.values]

years_train = df_model.loc[train_mask, "Year"].astype(int).values

folds_tune = make_walk_forward_folds(years_train, min_train_years=3)
folds_oof  = make_walk_forward_folds(years_train, min_train_years=2)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print("Train years:", sorted(df_model.loc[train_mask, "Year"].unique()))
print("Test years:", sorted(df_model.loc[test_mask, "Year"].unique()))
print("folds_tune:", len(folds_tune), "| folds_oof:", len(folds_oof))


# ============================================================
# STAGE 1: BASELINE
# ============================================================
print("\n====================")
print("STAGE 1: BASELINE")
print("====================")

lasso_cv_base = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("lasso_cv", LassoCV(cv=folds_tune, random_state=42, n_alphas=150, max_iter=100000)),
])

xgb_base = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("xgb", xgb.XGBRegressor(
        n_estimators=600,
        learning_rate=0.04,
        max_depth=3,                 # shallower for annual + generalization
        min_child_weight=10,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=5.0,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1,
        tree_method="hist",
        eval_metric="rmse",
    ))
])

mlp_base = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("mlp", MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        alpha=5e-3,                  # more regularization for small annual sample
        learning_rate_init=1e-3,
        max_iter=3000,
        early_stopping=True,
        validation_fraction=0.20,
        n_iter_no_change=30,
        random_state=42
    ))
])

baseline_models = {"lasso": lasso_cv_base, "xgb": xgb_base, "mlp": mlp_base}
base_perf, _ = fit_eval_models(baseline_models, X_train, y_train, X_test, y_test)
print(base_perf)

# --- Stacking baseline (need FIXED LASSO; no internal CV)
lasso_cv_base.fit(X_train, y_train)
alpha_base = lasso_cv_base.named_steps["lasso_cv"].alpha_

lasso_fixed_base = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=alpha_base, max_iter=200000, random_state=42)),
])

baseline_stack_fixed = {"lasso": lasso_fixed_base, "xgb": xgb_base, "mlp": mlp_base}

ens_tr, ens_te, valid_tr, w, info = stack_with_oof_walk_forward(
    baseline_stack_fixed, X_train, y_train, X_test, folds_oof
)

print("Stacking info (baseline):", info)
print_perf("BASELINE STACKED ENSEMBLE (walk-forward OOF Ridge)",
           y_train, ens_tr, y_test, ens_te, train_mask=valid_tr)
print("Baseline ensemble weights:\n", w.sort_values(ascending=False))


# ============================================================
# STAGE 2: REGULARIZED / TUNED
# ============================================================
print("\n====================")
print("STAGE 2: REGULARIZED / TUNED")
print("====================")

lasso_cv_reg = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("lasso_cv", LassoCV(cv=folds_tune, random_state=42, n_alphas=250, max_iter=200000)),
])

rf_reg = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("rf", RandomForestRegressor(
        n_estimators=800,
        max_depth=4,               # conservative depth to reduce overfit
        min_samples_leaf=20,
        min_samples_split=40,
        max_features=0.5,
        bootstrap=True,
        n_jobs=-1,
        random_state=42
    ))
])

xgb_tune_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("xgb", xgb.XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        tree_method="hist",
        eval_metric="rmse",
        n_jobs=-1
    ))
])

param_dist = {
    "xgb__n_estimators": np.arange(200, 1201, 100),
    "xgb__learning_rate": np.array([0.01, 0.02, 0.03, 0.05, 0.08]),
    "xgb__max_depth": np.arange(2, 6),
    "xgb__min_child_weight": np.array([5, 10, 15, 20]),
    "xgb__gamma": np.array([0.0, 0.1, 0.5, 1.0, 2.0]),
    "xgb__subsample": np.array([0.6, 0.7, 0.8, 0.9]),
    "xgb__colsample_bytree": np.array([0.6, 0.7, 0.8, 0.9]),
    "xgb__reg_alpha": np.array([0.0, 0.01, 0.05, 0.1, 0.5, 1.0]),
    "xgb__reg_lambda": np.array([1.0, 2.0, 5.0, 10.0, 20.0]),
}

xgb_search = RandomizedSearchCV(
    xgb_tune_pipe,
    param_distributions=param_dist,
    n_iter=50,
    scoring="neg_root_mean_squared_error",
    cv=folds_tune,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

print("\n--- Tuning XGBoost (train-only, time-aware CV) ---")
xgb_search.fit(X_train, y_train)
xgb_best = xgb_search.best_estimator_
best_params = xgb_search.best_params_
best_cv_rmse = -xgb_search.best_score_

print("Best CV RMSE:", round(best_cv_rmse, 4))
print("Best params:", best_params)

reg_models = {"lasso": lasso_cv_reg, "xgb": xgb_best, "rf": rf_reg}
reg_perf, _ = fit_eval_models(reg_models, X_train, y_train, X_test, y_test)
print(reg_perf)

# --- Regularized stacking: fixed LASSO + tuned XGB + RF
lasso_cv_reg.fit(X_train, y_train)
alpha_reg = lasso_cv_reg.named_steps["lasso_cv"].alpha_

lasso_fixed_reg = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=alpha_reg, max_iter=200000, random_state=42)),
])

reg_stack_fixed = {"lasso": lasso_fixed_reg, "xgb": xgb_best, "rf": rf_reg}

ens_tr2, ens_te2, valid_tr2, w2, info2 = stack_with_oof_walk_forward(
    reg_stack_fixed, X_train, y_train, X_test, folds_oof
)

print("Stacking info (regularized):", info2)
print_perf("REGULARIZED STACKED ENSEMBLE (walk-forward OOF Ridge)",
           y_train, ens_tr2, y_test, ens_te2, train_mask=valid_tr2)
print("Regularized ensemble weights:\n", w2.sort_values(ascending=False))


# ============================================================
# OPTIONAL: Rolling-origin robustness check (tuned XGB; no retuning)
# ============================================================
print("\n--- Rolling-origin robustness (tuned XGB; no retuning) ---")
roll = rolling_origin_eval(
    model=xgb_best,
    df_model=df_model,
    X_all=X,
    y_all=y,
    start_train_end=2017,
    end_year=2024
)
print(roll)

# ============================================================
# Back-transform metrics to original scale if log1p was used
# ============================================================
if USE_LOG_TARGET:
    # Use the regularized stacked model for reporting (common choice)
    # Train preds are OOF only where valid_tr2, so back-transform those slices.
    y_train_orig = np.expm1(y_train)
    y_test_orig  = np.expm1(y_test)
    pred_test_orig = np.expm1(ens_te2)

    print("\n--- TEST RESULTS (original scale; back-transformed) ---")
    print(eval_metrics(y_test_orig, pred_test_orig))

# ============================================================
# Print what to report
# ============================================================
final_xgb_params_to_report = {k.replace("xgb__", ""): v for k, v in best_params.items()}
print("\n=== FINAL XGB PARAMS TO REPORT ===")
print(final_xgb_params_to_report)

print("\n=== TUNING SUMMARY TO REPORT ===")
print({
    "target": target,
    "target_transform": "log1p" if USE_LOG_TARGET else "none",
    "tuning_method": "RandomizedSearchCV",
    "cv_design": "Walk-forward (expanding window) folds by year within training period (<=2019)",
    "selection_metric": "CV RMSE (model scale)",
    "best_cv_rmse": float(best_cv_rmse),
    "n_iter": 50,
    "stacking_note": "Meta-learner trained on walk-forward OOF predictions; earliest year(s) have no prior data and are excluded."
})


Final feature count: 36
Total rows for modeling: 539
Train size: 297, Test size: 242
Train years: [2014, 2015, 2016, 2017, 2018, 2019]
Test years: [2020, 2021, 2022, 2023, 2024]
folds_tune: 3 | folds_oof: 4

STAGE 1: BASELINE
   model  r2_train  rmse_train  mae_train     r2_test  rmse_test  mae_test
1    xgb  0.986614    0.015550   0.009010    0.471374   0.109610  0.080481
0  lasso  0.785818    0.062200   0.047760   -0.065036   0.155582  0.124222
2    mlp -0.837078    0.182164   0.093459 -510.536862   3.409691  3.092839
Stacking info (baseline): {'meta_train_kept': 197, 'meta_train_dropped': 100, 'meta_alpha': 0.001}

BASELINE STACKED ENSEMBLE (walk-forward OOF Ridge)
  Train: R2=0.711, RMSE=0.068, MAE=0.041   (n=197)
  Test : R2=0.397, RMSE=0.117, MAE=0.086   (n=242)
Baseline ensemble weights:
 pred_xgb      0.623365
pred_lasso    0.310695
pred_mlp     -0.000039
dtype: float64

STAGE 2: REGULARIZED / TUNED

--- Tuning XGBoost (train-only, time-aware CV) ---
Best CV RMSE: 0.0455
Best p

In [37]:
# ============================================================
# STATE-YEAR HIRING MODEL (A–E IMPLEMENTATION)
#  A) Add lag1/lag2 (incl. lagged target) - NOT leakage if built via groupby shift
#  B) Model relative-to-US: y_rel = log1p(state_hires_per_1k) - log1p(US_hires_per_1k)
#  C) MERF with random slopes (Z includes intercept + US macro + time trend)
#  D) Bring in stronger economic predictors (lagged emp_per_1k, unemp, earnings, etc.)
#  E) Stop chasing stacking; compare against strong baselines (persistence + state pre-avg rel)
# ============================================================

import numpy as np
import pandas as pd
import warnings
import sys
import os

from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor

from merf.merf import MERF

warnings.filterwarnings("ignore")


# ============================================================
# CONFIG
# ============================================================
START_YEAR = 2014
END_YEAR   = 2024
TRAIN_END_YEAR = 2019

EXCLUDE_STATES = {"Alaska"}   # optional
RANDOM_STATE = 42

# Raw target (state hires per 1k)
TARGET_RAW = "econ_hires_per_1k"

# Needed to compute US_hires_per_1k
STATE_HIRES_TOTAL_COL = "state_hires_total"
STATE_POP_COL         = "state_total_pop"

# Lags
LAGS = [1, 2]  # "at most lag 2" per your request

# MERF / RF tuning and final fit
TUNE_N_ESTIMATORS  = 200
FINAL_N_ESTIMATORS = 900
MERF_TUNE_ITERS    = 6
MERF_FINAL_ITERS   = 25

# Walk-forward tuning folds within train window (<=2019)
MIN_TRAIN_YEARS_TUNE = 3

# If you want a stricter "forecasting" stance for slow variables,
# set USE_LAGGED_SLOW_VARS=True to only use lagged versions of health/social vars.
USE_LAGGED_SLOW_VARS = False


# ============================================================
# HELPERS
# ============================================================
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")
    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            sys.stdout.close()
        finally:
            sys.stdout = self._original_stdout

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def metrics_block(name, y_true, y_pred):
    return {
        "split": name,
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": rmse(y_true, y_pred),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "n": int(len(y_true)),
    }

def make_walk_forward_splits_by_year(years: np.ndarray, min_train_years: int = 3):
    years = np.asarray(years).astype(int)
    uniq = np.sort(np.unique(years))
    if len(uniq) < min_train_years + 1:
        raise ValueError("Not enough unique years for walk-forward folds.")
    splits = []
    for i in range(min_train_years, len(uniq)):
        tr_years = uniq[:i]
        va_year  = uniq[i]
        tr_idx = np.where(np.isin(years, tr_years))[0]
        va_idx = np.where(years == va_year)[0]
        if len(tr_idx) and len(va_idx):
            splits.append((tr_idx, va_idx))
    if not splits:
        raise ValueError("No walk-forward splits created.")
    return splits

def safe_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def add_us_macro(df):
    """
    Compute US_hires_per_1k by year from state totals.
    """
    if (STATE_HIRES_TOTAL_COL not in df.columns) or (STATE_POP_COL not in df.columns):
        raise ValueError("Missing columns for US macro: state_hires_total and/or state_total_pop.")

    df = safe_numeric(df, [STATE_HIRES_TOTAL_COL, STATE_POP_COL])
    us_macro = (
        df.groupby("Year", as_index=False)
          .agg(US_hires_sum=(STATE_HIRES_TOTAL_COL, "sum"),
               US_pop_sum=(STATE_POP_COL, "sum"))
    )
    us_macro["US_hires_per_1k"] = 1000.0 * us_macro["US_hires_sum"] / us_macro["US_pop_sum"].replace({0: np.nan})
    return df.merge(us_macro[["Year", "US_hires_per_1k"]], on="Year", how="left")

def add_lags_by_state(df, cols, lags=(1,2)):
    """
    Create lag features strictly within each State, by Year order.
    """
    df = df.sort_values(["State", "Year"]).copy()
    for col in cols:
        if col not in df.columns:
            continue
        for L in lags:
            df[f"{col}_lag{L}"] = df.groupby("State")[col].shift(L)
    return df

def reconstruct_hires_from_rel(pred_y_rel, us_hires_per_1k):
    """
    y_rel = log1p(state_hires_per_1k) - log1p(US_hires_per_1k)
    => log1p(state_hires) = y_rel + log1p(US_hires)
    => state_hires = expm1(...)
    """
    pred_log1p_hires = pred_y_rel + np.log1p(us_hires_per_1k)
    return np.expm1(pred_log1p_hires)

def compute_y_rel(df):
    """
    y_rel = log1p(state_hires_per_1k) - log1p(US_hires_per_1k)
    """
    y_rel = np.log1p(df[TARGET_RAW].astype(float)) - np.log1p(df["US_hires_per_1k"].astype(float))
    return y_rel

def baseline_persistence(df, kind="absolute"):
    """
    kind="absolute": predict hires_t = hires_{t-1}
    kind="relative": predict y_rel_t = y_rel_{t-1} (using lagged y_rel)
    """
    if kind == "absolute":
        return df[f"{TARGET_RAW}_lag1"].to_numpy(dtype=float)
    elif kind == "relative":
        # build y_rel lag1 from its components:
        # y_rel_lag1 = log1p(hires_lag1) - log1p(US_hires_lag1)
        y_rel_lag1 = np.log1p(df[f"{TARGET_RAW}_lag1"].astype(float)) - np.log1p(df["US_hires_per_1k_lag1"].astype(float))
        return y_rel_lag1.to_numpy(dtype=float)
    else:
        raise ValueError("kind must be 'absolute' or 'relative'")

def baseline_state_preavg_rel(df_train, df_any):
    """
    Baseline: for each state, use its average y_rel in training period (<= TRAIN_END_YEAR),
    then reconstruct hires with that constant rel + current US_hires_per_1k.
    """
    tmp = df_train.copy()
    tmp["y_rel"] = compute_y_rel(tmp)
    state_rel_mean = tmp.groupby("State")["y_rel"].mean()

    rel_hat = df_any["State"].map(state_rel_mean).to_numpy(dtype=float)
    # if any state missing (shouldn't), fall back to overall mean
    if np.any(~np.isfinite(rel_hat)):
        rel_hat[~np.isfinite(rel_hat)] = float(np.nanmean(rel_hat))
    return rel_hat


# ============================================================
# 1) LOAD / CLEAN
# ============================================================
df0 = df1.copy()

# drop index-like columns
for c in ["Unnamed: 0", "Unnamed: 0.1", "index"]:
    if c in df0.columns:
        df0 = df0.drop(columns=[c])

# basic checks
if "State" not in df0.columns or "Year" not in df0.columns:
    raise ValueError("df1 must contain at least columns: ['State','Year'].")

df0["State"] = df0["State"].astype(str)
df0["Year"]  = pd.to_numeric(df0["Year"], errors="coerce")
df0 = df0.dropna(subset=["State", "Year"]).copy()
df0["Year"]  = df0["Year"].astype(int)

df0 = df0[(df0["Year"] >= START_YEAR) & (df0["Year"] <= END_YEAR)].copy()
if EXCLUDE_STATES:
    df0 = df0[~df0["State"].isin(EXCLUDE_STATES)].copy()

# numeric
df0 = safe_numeric(df0, [TARGET_RAW, "econ_emp_per_1k", "state_avg_earnings", "STATE_% Unemployed", STATE_HIRES_TOTAL_COL, STATE_POP_COL])

# drop missing target
df0 = df0.dropna(subset=[TARGET_RAW]).copy()

# US macro
df0 = add_us_macro(df0)
df0 = df0.dropna(subset=["US_hires_per_1k"]).copy()

# time features
df0["t_trend"] = df0["Year"] - df0["Year"].min()
df0["post_2020"] = (df0["Year"] >= 2020).astype(int)

# lag US macro too (needed for relative persistence baseline)
df0 = add_lags_by_state(df0, cols=["US_hires_per_1k"], lags=LAGS)  # US series is year-level, but we lag per state for alignment

# ============================================================
# 2) ADD LAGS (A) — including lagged target
# ============================================================
lag_cols_core = [
    TARGET_RAW,
    "econ_emp_per_1k",
    "STATE_% Unemployed",
    "state_avg_earnings",
]
df0 = add_lags_by_state(df0, cols=lag_cols_core, lags=LAGS)

# Optional: lag some health/social vars (you can expand if desired)
health_social = [
    "STATE_% Adults with Obesity",
    "STATE_% Children in Poverty",
    "STATE_% Excessive Drinking",
    "STATE_% Fair or Poor Health",
    "STATE_% Smokers",
    "STATE_Food Environment Index",
    "STATE_Income Ratio",
    "STATE_Mentally Unhealthy Days",
    "STATE_Physically Unhealthy Days",
    "STATE_Primary Care Physicians Rate",
    "STATE_Social Association Rate",
    "STATE_Violent Crime Rate",
]
# Ensure numeric
df0 = safe_numeric(df0, health_social)

# If you want strict forecasting with slow vars, use lagged versions; else keep levels
if USE_LAGGED_SLOW_VARS:
    df0 = add_lags_by_state(df0, cols=health_social, lags=LAGS)

# Simple change features (often helps with regime shifts)
for col in ["STATE_% Unemployed", "state_avg_earnings", "econ_emp_per_1k"]:
    c1 = f"{col}_lag1"
    c2 = f"{col}_lag2"
    if c1 in df0.columns and c2 in df0.columns:
        df0[f"d_{col}_lag1"] = df0[c1] - df0[c2]

# Drop rows without lag1 for key series (2014 will drop; that’s expected)
required_for_model = [f"{TARGET_RAW}_lag1", "US_hires_per_1k", "US_hires_per_1k_lag1"]
df_model = df0.dropna(subset=required_for_model).copy()

print("Rows after lag requirements:", df_model.shape[0])
print("Years in model:", sorted(df_model["Year"].unique()))

# ============================================================
# 3) TARGET (B) — relative to US (log-difference)
# ============================================================
df_model["y_rel"] = compute_y_rel(df_model)

# ============================================================
# 4) FEATURE SET (D) — stronger econ + reasonable health/social
# ============================================================
X_cols = []

# Strong dynamics
for c in [
    f"{TARGET_RAW}_lag1", f"{TARGET_RAW}_lag2",
    "econ_emp_per_1k_lag1", "econ_emp_per_1k_lag2",
    "STATE_% Unemployed_lag1", "STATE_% Unemployed_lag2",
    "state_avg_earnings_lag1", "state_avg_earnings_lag2",
    "d_STATE_% Unemployed_lag1", "d_state_avg_earnings_lag1", "d_econ_emp_per_1k_lag1",
]:
    if c in df_model.columns:
        X_cols.append(c)

# National macro and time structure (fixed effects)
for c in ["US_hires_per_1k", "t_trend", "post_2020"]:
    if c in df_model.columns:
        X_cols.append(c)

# Health/social
if USE_LAGGED_SLOW_VARS:
    for base in health_social:
        for L in LAGS:
            c = f"{base}_lag{L}"
            if c in df_model.columns:
                X_cols.append(c)
else:
    for c in health_social:
        if c in df_model.columns:
            X_cols.append(c)

# Deduplicate
X_cols = list(dict.fromkeys(X_cols))
if not X_cols:
    raise ValueError("No features found. Check column names / lag creation.")

print("Feature count:", len(X_cols))

# ============================================================
# 5) SPLIT
# ============================================================
df_tr = df_model[df_model["Year"] <= TRAIN_END_YEAR].copy()
df_te = df_model[df_model["Year"] >  TRAIN_END_YEAR].copy()

# MERF needs clusters in test to exist in train
train_states = set(df_tr["State"].unique())
df_te = df_te[df_te["State"].isin(train_states)].copy()

y_tr = df_tr["y_rel"].to_numpy(dtype=float)
y_te = df_te["y_rel"].to_numpy(dtype=float)

cl_tr = df_tr["State"].astype(str)  # Series
cl_te = df_te["State"].astype(str)

# (C) Random effects design matrix Z: intercept + US macro + time trend
Z_tr = np.column_stack([
    np.ones(len(df_tr), dtype=float),
    df_tr["US_hires_per_1k"].to_numpy(dtype=float),
    df_tr["t_trend"].to_numpy(dtype=float),
])
Z_te = np.column_stack([
    np.ones(len(df_te), dtype=float),
    df_te["US_hires_per_1k"].to_numpy(dtype=float),
    df_te["t_trend"].to_numpy(dtype=float),
])

X_tr_raw = df_tr[X_cols].copy()
X_te_raw = df_te[X_cols].copy()

years_tr = df_tr["Year"].astype(int).to_numpy()
wf_splits = make_walk_forward_splits_by_year(years_tr, min_train_years=MIN_TRAIN_YEARS_TUNE)

print(f"Train n={len(df_tr)} | Test n={len(df_te)}")
print("Train years:", sorted(df_tr["Year"].unique()))
print("Test years :", sorted(df_te["Year"].unique()))
print("Walk-forward tuning folds:", len(wf_splits))


# ============================================================
# 6) BASELINES (E)
# ============================================================
print("\n====================")
print("BASELINES (original scale)")
print("====================")

# Persistence baseline: hires_t ≈ hires_{t-1}
pred_te_persist_abs = df_te[f"{TARGET_RAW}_lag1"].to_numpy(dtype=float)
y_te_abs = df_te[TARGET_RAW].to_numpy(dtype=float)
print(metrics_block("TEST | Persistence hires(t-1)", y_te_abs, pred_te_persist_abs))

# Relative-to-US state pre-avg baseline (constant per state, trained on <=2019)
rel_hat_te = baseline_state_preavg_rel(df_tr, df_te)
pred_te_preavg_abs = reconstruct_hires_from_rel(rel_hat_te, df_te["US_hires_per_1k"].to_numpy(dtype=float))
print(metrics_block("TEST | State pre-avg rel + US", y_te_abs, pred_te_preavg_abs))


# ============================================================
# 7) TUNE MERF (RF fixed effects) with walk-forward CV
# ============================================================
print("\n====================")
print("TUNING MERF (walk-forward CV on y_rel)")
print("====================")

param_grid = {
    "max_depth": [3, 4, 5, 6],
    "min_samples_leaf": [10, 20, 30],
    "min_samples_split": [20, 40, 60],
    "max_features": [0.4, 0.6, "sqrt"],
}

best_rmse = np.inf
best_params = None

with HiddenPrints():
    for params in ParameterGrid(param_grid):
        fold_scores = []

        for tr_idx, va_idx in wf_splits:
            X_f_tr_raw = X_tr_raw.iloc[tr_idx]
            y_f_tr     = y_tr[tr_idx]
            Z_f_tr     = Z_tr[tr_idx]
            cl_f_tr    = cl_tr.iloc[tr_idx]

            X_f_va_raw = X_tr_raw.iloc[va_idx]
            y_f_va     = y_tr[va_idx]
            Z_f_va     = Z_tr[va_idx]
            cl_f_va    = cl_tr.iloc[va_idx]

            imp = SimpleImputer(strategy="median")
            X_f_tr = imp.fit_transform(X_f_tr_raw)
            X_f_va = imp.transform(X_f_va_raw)

            rf = RandomForestRegressor(
                n_estimators=TUNE_N_ESTIMATORS,
                n_jobs=-1,
                random_state=RANDOM_STATE,
                **params
            )
            m = MERF(rf, max_iterations=MERF_TUNE_ITERS)
            m.fit(X_f_tr, Z_f_tr, cl_f_tr, y_f_tr)

            p_va = m.predict(X_f_va, Z_f_va, cl_f_va)
            fold_scores.append(rmse(y_f_va, p_va))

        avg_score = float(np.mean(fold_scores))
        if avg_score < best_rmse:
            best_rmse = avg_score
            best_params = params

print("Best CV RMSE (y_rel scale):", round(best_rmse, 5))
print("Best RF params:", best_params)


# ============================================================
# 8) FINAL MERF FIT + EVAL (y_rel -> hires per 1k)
# ============================================================
print("\n====================")
print("FINAL MERF FIT + EVAL (report on original scale)")
print("====================")

imp_final = SimpleImputer(strategy="median")
X_tr = imp_final.fit_transform(X_tr_raw)
X_te = imp_final.transform(X_te_raw)

rf_final = RandomForestRegressor(
    n_estimators=FINAL_N_ESTIMATORS,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    **(best_params if best_params is not None else {})
)

with HiddenPrints():
    merf_final = MERF(rf_final, max_iterations=MERF_FINAL_ITERS)
    merf_final.fit(X_tr, Z_tr, cl_tr, y_tr)

pred_tr_rel = merf_final.predict(X_tr, Z_tr, cl_tr)
pred_te_rel = merf_final.predict(X_te, Z_te, cl_te)

# Convert back to hires per 1k for evaluation
y_tr_abs = df_tr[TARGET_RAW].to_numpy(dtype=float)
y_te_abs = df_te[TARGET_RAW].to_numpy(dtype=float)

pred_tr_abs = reconstruct_hires_from_rel(pred_tr_rel, df_tr["US_hires_per_1k"].to_numpy(dtype=float))
pred_te_abs = reconstruct_hires_from_rel(pred_te_rel, df_te["US_hires_per_1k"].to_numpy(dtype=float))

print(metrics_block("TRAIN | MERF(rel->abs)", y_tr_abs, pred_tr_abs))
print(metrics_block("TEST  | MERF(rel->abs)", y_te_abs, pred_te_abs))

# Fixed-effect RF importances (interpret as "within-year/state features", not causal)
try:
    imp_df = pd.DataFrame({
        "feature": X_cols,
        "importance": merf_final.trained_fe_model.feature_importances_
    }).sort_values("importance", ascending=False)

    print("\nTop 20 fixed-effect drivers (RF in MERF):")
    print(imp_df.head(20).to_string(index=False))
except Exception as e:
    print("\nFeature importances not available:", repr(e))


# ============================================================
# 9) ROLLING-ORIGIN BACKTEST 2020..2024 (EVALUATE IN ABS SCALE)
# ============================================================
print("\n====================")
print("ROLLING-ORIGIN BACKTEST (2020..2024) | MERF(rel->abs)")
print("====================")

def rolling_origin_merf(df_model, start_test_year=2020, end_test_year=2024,
                        x_cols=None, use_params=None):
    rows = []

    for test_year in range(start_test_year, end_test_year + 1):
        df_tr_y = df_model[df_model["Year"] <= (test_year - 1)].copy()
        df_te_y = df_model[df_model["Year"] == test_year].copy()

        # keep only states seen in train
        seen = set(df_tr_y["State"].unique())
        df_te_y = df_te_y[df_te_y["State"].isin(seen)].copy()

        if df_tr_y.empty or df_te_y.empty:
            continue

        y_tr_rel_y = compute_y_rel(df_tr_y).to_numpy(dtype=float)
        y_te_rel_y = compute_y_rel(df_te_y).to_numpy(dtype=float)

        cl_tr_y = df_tr_y["State"].astype(str)
        cl_te_y = df_te_y["State"].astype(str)

        Z_tr_y = np.column_stack([
            np.ones(len(df_tr_y), dtype=float),
            df_tr_y["US_hires_per_1k"].to_numpy(dtype=float),
            df_tr_y["t_trend"].to_numpy(dtype=float),
        ])
        Z_te_y = np.column_stack([
            np.ones(len(df_te_y), dtype=float),
            df_te_y["US_hires_per_1k"].to_numpy(dtype=float),
            df_te_y["t_trend"].to_numpy(dtype=float),
        ])

        X_tr_y_raw = df_tr_y[x_cols].copy()
        X_te_y_raw = df_te_y[x_cols].copy()

        imp = SimpleImputer(strategy="median")
        X_tr_y = imp.fit_transform(X_tr_y_raw)
        X_te_y = imp.transform(X_te_y_raw)

        rf = RandomForestRegressor(
            n_estimators=FINAL_N_ESTIMATORS,
            n_jobs=-1,
            random_state=RANDOM_STATE,
            **(use_params if use_params is not None else {})
        )
        m = MERF(rf, max_iterations=MERF_FINAL_ITERS)
        with HiddenPrints():
            m.fit(X_tr_y, Z_tr_y, cl_tr_y, y_tr_rel_y)

        pred_te_rel_y = m.predict(X_te_y, Z_te_y, cl_te_y)

        # evaluate on original hires per 1k
        y_true_abs = df_te_y[TARGET_RAW].to_numpy(dtype=float)
        y_pred_abs = reconstruct_hires_from_rel(pred_te_rel_y, df_te_y["US_hires_per_1k"].to_numpy(dtype=float))

        rows.append({
            "test_year": test_year,
            **metrics_block(f"YEAR_{test_year}", y_true_abs, y_pred_abs)
        })

    return pd.DataFrame(rows)

bt = rolling_origin_merf(df_model=df_model, start_test_year=2020, end_test_year=2024,
                         x_cols=X_cols, use_params=best_params)

if bt.empty:
    print("No backtest rows produced (check data coverage).")
else:
    print(bt[["test_year", "r2", "rmse", "mae", "n"]].to_string(index=False))


# ============================================================
# 10) OPTIONAL: What to use for state-level case studies
# ============================================================
# For case studies, create a tidy frame with:
#  - actual hires per 1k
#  - predicted hires per 1k
#  - predicted relative deviation (y_rel) and actual y_rel
#
# Example (on TEST set):
case_te = df_te[["State", "Year", TARGET_RAW, "US_hires_per_1k"]].copy()
case_te["y_rel_true"] = y_te
case_te["y_rel_pred"] = pred_te_rel
case_te["hires_pred_per_1k"] = pred_te_abs
case_te["excess_hires_per_1k_vs_US"] = case_te["hires_pred_per_1k"] - case_te["US_hires_per_1k"]

print("\nCase-study table (head):")
print(case_te.head(10).to_string(index=False))


Rows after lag requirements: 487
Years in model: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Feature count: 26
Train n=245 | Test n=242
Train years: [2015, 2016, 2017, 2018, 2019]
Test years : [2020, 2021, 2022, 2023, 2024]
Walk-forward tuning folds: 2

BASELINES (original scale)
{'split': 'TEST | Persistence hires(t-1)', 'r2': 0.42152767523947987, 'rmse': 34.131393024209736, 'mae': 26.911492517779255, 'n': 242}
{'split': 'TEST | State pre-avg rel + US', 'r2': 0.8095372012016702, 'rmse': 19.58473960135387, 'mae': 13.466087818137826, 'n': 242}

TUNING MERF (walk-forward CV on y_rel)
Best CV RMSE (y_rel scale): 0.03827
Best RF params: {'max_depth': 6, 'max_features': 0.6, 'min_samples_leaf': 10, 'min_samples_split': 20}

FINAL MERF FIT + EVAL (report on original scale)
{'split': 'TRAIN | MERF(rel->abs)', 'r2': 0.9912707683666939, 'rmse': 3.6881456056286805, 'mae': 2.319628862930542, 'n': 245}
{'split': 'TEST  | MERF(rel->abs)', 'r2': 0.5558472638393466, 'rmse': 29.907415

In [ ]:
# ============================================================
# STATE-YEAR HIRING MODEL (Improved)
# - Test only: 2023–2024; Train: 2015–2022
# - Target: y_rel = log1p(state_hires_per_1k) - log1p(US_hires_per_1k)
# - Strong baseline: (shrunken) state mean y_rel + current US
# - Improvement: model residual with regularized AR(2) + econ controls (Ridge)
# - Optional: MERF using y_rel lags (avoid absolute target lags)
# ============================================================

import numpy as np
import pandas as pd
import warnings
import sys
import os

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor

from merf.merf import MERF

warnings.filterwarnings("ignore")


# ----------------------------
# CONFIG
# ----------------------------
START_YEAR = 2014
END_YEAR   = 2024
TEST_YEARS = [2023, 2024]         # <<<<<<<<<<<< requested
EXCLUDE_STATES = {"Alaska"}       # optional
RANDOM_STATE = 42

TARGET_RAW = "econ_hires_per_1k"
STATE_HIRES_TOTAL_COL = "state_hires_total"
STATE_POP_COL         = "state_total_pop"

LAGS = [1, 2]                     # at most lag 2

# Baseline shrinkage (helps small-sample states; usually mild effect here)
# shrunk_mean = (n/(n+K))*state_mean + (K/(n+K))*global_mean
SHRINK_K = 2.0

# RidgeCV grid
RIDGE_ALPHAS = np.logspace(-3, 4, 30)

# MERF settings (optional)
RUN_MERF = True
TUNE_N_ESTIMATORS  = 200
FINAL_N_ESTIMATORS = 900
MERF_TUNE_ITERS    = 6
MERF_FINAL_ITERS   = 25
MIN_TRAIN_YEARS_TUNE = 3


# ----------------------------
# Helpers
# ----------------------------
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")
    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            sys.stdout.close()
        finally:
            sys.stdout = self._original_stdout

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def metrics_block(name, y_true, y_pred):
    return {
        "split": name,
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": rmse(y_true, y_pred),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "n": int(len(y_true)),
    }

def safe_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def add_us_macro(df):
    if (STATE_HIRES_TOTAL_COL not in df.columns) or (STATE_POP_COL not in df.columns):
        raise ValueError("Missing columns for US macro: state_hires_total and/or state_total_pop.")
    df = safe_numeric(df, [STATE_HIRES_TOTAL_COL, STATE_POP_COL])

    us_macro = (
        df.groupby("Year", as_index=False)
          .agg(US_hires_sum=(STATE_HIRES_TOTAL_COL, "sum"),
               US_pop_sum=(STATE_POP_COL, "sum"))
    )
    us_macro["US_hires_per_1k"] = 1000.0 * us_macro["US_hires_sum"] / us_macro["US_pop_sum"].replace({0: np.nan})
    return df.merge(us_macro[["Year", "US_hires_per_1k"]], on="Year", how="left")

def add_lags_by_state(df, cols, lags=(1,2)):
    df = df.sort_values(["State", "Year"]).copy()
    for col in cols:
        if col not in df.columns:
            continue
        for L in lags:
            df[f"{col}_lag{L}"] = df.groupby("State")[col].shift(L)
    return df

def compute_y_rel(df):
    return np.log1p(df[TARGET_RAW].astype(float)) - np.log1p(df["US_hires_per_1k"].astype(float))

def compute_y_rel_from_lags(df, L):
    # y_rel_lagL = log1p(hires_lagL) - log1p(US_lagL)
    return np.log1p(df[f"{TARGET_RAW}_lag{L}"].astype(float)) - np.log1p(df[f"US_hires_per_1k_lag{L}"].astype(float))

def reconstruct_hires_from_rel(pred_y_rel, us_hires_per_1k):
    pred_log1p = pred_y_rel + np.log1p(us_hires_per_1k)
    return np.expm1(pred_log1p)

def make_walk_forward_splits_by_year(years: np.ndarray, min_train_years: int = 3):
    years = np.asarray(years).astype(int)
    uniq = np.sort(np.unique(years))
    if len(uniq) < min_train_years + 1:
        raise ValueError("Not enough unique years for walk-forward folds.")
    splits = []
    for i in range(min_train_years, len(uniq)):
        tr_years = uniq[:i]
        va_year  = uniq[i]
        tr_idx = np.where(np.isin(years, tr_years))[0]
        va_idx = np.where(years == va_year)[0]
        if len(tr_idx) and len(va_idx):
            splits.append((tr_idx, va_idx))
    if not splits:
        raise ValueError("No walk-forward splits created.")
    return splits

def build_shrunken_state_means(df_train):
    tmp = df_train.copy()
    tmp["y_rel"] = compute_y_rel(tmp)

    gmean = float(tmp["y_rel"].mean())
    stats = tmp.groupby("State")["y_rel"].agg(["mean", "count"]).rename(columns={"mean":"state_mean", "count":"n"})
    stats["shrunk_mean"] = (stats["n"]/(stats["n"] + SHRINK_K))*stats["state_mean"] + (SHRINK_K/(stats["n"] + SHRINK_K))*gmean
    return stats["shrunk_mean"], gmean


# ============================================================
# 1) LOAD / CLEAN
# ============================================================
df0 = df1.copy()

for c in ["Unnamed: 0", "Unnamed: 0.1", "index"]:
    if c in df0.columns:
        df0 = df0.drop(columns=[c])

if "State" not in df0.columns or "Year" not in df0.columns:
    raise ValueError("df1 must contain at least columns: ['State','Year'].")

df0["State"] = df0["State"].astype(str)
df0["Year"]  = pd.to_numeric(df0["Year"], errors="coerce")
df0 = df0.dropna(subset=["State", "Year"]).copy()
df0["Year"]  = df0["Year"].astype(int)

df0 = df0[(df0["Year"] >= START_YEAR) & (df0["Year"] <= END_YEAR)].copy()
if EXCLUDE_STATES:
    df0 = df0[~df0["State"].isin(EXCLUDE_STATES)].copy()

df0 = safe_numeric(df0, [TARGET_RAW, "econ_emp_per_1k", "state_avg_earnings", "STATE_% Unemployed",
                         STATE_HIRES_TOTAL_COL, STATE_POP_COL])

df0 = df0.dropna(subset=[TARGET_RAW]).copy()

# US macro + time features
df0 = add_us_macro(df0)
df0 = df0.dropna(subset=["US_hires_per_1k"]).copy()

df0["t_trend"] = df0["Year"] - df0["Year"].min()
df0["post_2020"] = (df0["Year"] >= 2020).astype(int)

# Lags (target + econ + US macro)
lag_cols = [TARGET_RAW, "econ_emp_per_1k", "STATE_% Unemployed", "state_avg_earnings", "US_hires_per_1k"]
df0 = add_lags_by_state(df0, lag_cols, lags=LAGS)

# Build y_rel + its lags
df0["y_rel"] = compute_y_rel(df0)
for L in LAGS:
    df0[f"y_rel_lag{L}"] = compute_y_rel_from_lags(df0, L)

# Simple deltas on econ series (lag1 - lag2)
for base in ["econ_emp_per_1k", "STATE_% Unemployed", "state_avg_earnings"]:
    c1, c2 = f"{base}_lag1", f"{base}_lag2"
    if c1 in df0.columns and c2 in df0.columns:
        df0[f"d_{base}_lag1"] = df0[c1] - df0[c2]

# Keep rows where lag1 exists (2014 drops)
required = ["y_rel", "y_rel_lag1", "US_hires_per_1k"]
df_model = df0.dropna(subset=required).copy()

print("Rows after lag requirements:", df_model.shape[0])
print("Years in model:", sorted(df_model["Year"].unique()))

# ============================================================
# 2) TRAIN/TEST split (train = all except 2023/2024; test = 2023/2024)
# ============================================================
train_mask = ~df_model["Year"].isin(TEST_YEARS)
test_mask  =  df_model["Year"].isin(TEST_YEARS)

df_tr = df_model.loc[train_mask].copy()
df_te = df_model.loc[test_mask].copy()

# Make sure test states exist in train
seen = set(df_tr["State"].unique())
df_te = df_te[df_te["State"].isin(seen)].copy()

print(f"Train n={len(df_tr)} | Test n={len(df_te)}")
print("Train years:", sorted(df_tr["Year"].unique()))
print("Test years :", sorted(df_te["Year"].unique()))

# ============================================================
# 3) BASELINES (original scale)
# ============================================================
print("\n====================")
print("BASELINES (original scale)")
print("====================")

y_te_abs = df_te[TARGET_RAW].to_numpy(dtype=float)

# Baseline 1: Persistence in absolute space: hires_t ≈ hires_{t-1}
pred_te_persist_abs = df_te[f"{TARGET_RAW}_lag1"].to_numpy(dtype=float)
print(metrics_block("TEST | Persistence hires(t-1)", y_te_abs, pred_te_persist_abs))

# Baseline 2: State (shrunken) mean y_rel from TRAIN + current US
state_rel_shrunk, global_rel_mean = build_shrunken_state_means(df_tr)
rel_hat_te = df_te["State"].map(state_rel_shrunk).to_numpy(dtype=float)
missing = ~np.isfinite(rel_hat_te)
if missing.any():
    rel_hat_te[missing] = global_rel_mean

pred_te_preavg_abs = reconstruct_hires_from_rel(rel_hat_te, df_te["US_hires_per_1k"].to_numpy(dtype=float))
print(metrics_block("TEST | State shrunk mean y_rel + US", y_te_abs, pred_te_preavg_abs))

# ============================================================
# 4) RESIDUAL RIDGE MODEL (recommended)
#    y_rel = state_baseline + residual
#    residual ~ Ridge( y_rel_lag1_resid, y_rel_lag2_resid, econ lags/deltas, health/social )
# ============================================================
print("\n====================")
print("RESIDUAL RIDGE AR(2) around state baseline (report on original scale)")
print("====================")

# Build baseline per row
df_tr["rel_base"] = df_tr["State"].map(state_rel_shrunk).astype(float)
df_te["rel_base"] = df_te["State"].map(state_rel_shrunk).astype(float)
df_tr["rel_base"] = df_tr["rel_base"].fillna(global_rel_mean)
df_te["rel_base"] = df_te["rel_base"].fillna(global_rel_mean)

# Residual target
df_tr["rel_resid_y"] = df_tr["y_rel"] - df_tr["rel_base"]

# Residual lag features (encourage mean reversion around baseline)
df_tr["rel_resid_lag1"] = df_tr["y_rel_lag1"] - df_tr["rel_base"]
df_tr["rel_resid_lag2"] = df_tr["y_rel_lag2"] - df_tr["rel_base"] if "y_rel_lag2" in df_tr.columns else np.nan

df_te["rel_resid_lag1"] = df_te["y_rel_lag1"] - df_te["rel_base"]
df_te["rel_resid_lag2"] = df_te["y_rel_lag2"] - df_te["rel_base"] if "y_rel_lag2" in df_te.columns else np.nan

# Candidate predictors for residual
health_social = [
    "STATE_% Adults with Obesity",
    "STATE_% Children in Poverty",
    "STATE_% Excessive Drinking",
    "STATE_% Fair or Poor Health",
    "STATE_% Smokers",
    "STATE_Food Environment Index",
    "STATE_Income Ratio",
    "STATE_Mentally Unhealthy Days",
    "STATE_Physically Unhealthy Days",
    "STATE_Primary Care Physicians Rate",
    "STATE_Social Association Rate",
    "STATE_Violent Crime Rate",
]

df_model = safe_numeric(df_model, health_social)

ridge_feats = [
    "rel_resid_lag1", "rel_resid_lag2",
    "econ_emp_per_1k_lag1", "econ_emp_per_1k_lag2", "d_econ_emp_per_1k_lag1",
    "STATE_% Unemployed_lag1", "STATE_% Unemployed_lag2", "d_STATE_% Unemployed_lag1",
    "state_avg_earnings_lag1", "state_avg_earnings_lag2", "d_state_avg_earnings_lag1",
    "t_trend", "post_2020",
] + [c for c in health_social if c in df_tr.columns]

ridge_feats = [c for c in ridge_feats if c in df_tr.columns and c in df_te.columns]

X_tr_r = df_tr[ridge_feats].copy()
X_te_r = df_te[ridge_feats].copy()

y_tr_r = df_tr["rel_resid_y"].to_numpy(dtype=float)

ridge = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("ridge", RidgeCV(alphas=RIDGE_ALPHAS))
])
ridge.fit(X_tr_r, y_tr_r)

pred_te_resid = ridge.predict(X_te_r)
pred_te_rel = df_te["rel_base"].to_numpy(dtype=float) + pred_te_resid
pred_te_abs = reconstruct_hires_from_rel(pred_te_rel, df_te["US_hires_per_1k"].to_numpy(dtype=float))

print(metrics_block("TEST | Residual Ridge(rel->abs)", y_te_abs, pred_te_abs))
print("Ridge alpha:", float(ridge.named_steps["ridge"].alpha_))

# Optional: coefficients for interpretation (signs are meaningful after scaling; for raw units, refit w/o scaler)
coef = pd.Series(ridge.named_steps["ridge"].coef_, index=ridge_feats).sort_values(key=np.abs, ascending=False)
print("\nTop 15 Ridge coefficients by absolute magnitude:")
print(coef.head(15).to_string())

# ============================================================
# 5) OPTIONAL: MERF (compare)
# ============================================================
if RUN_MERF:
    print("\n====================")
    print("MERF (compare) using y_rel target + y_rel lags (report on original scale)")
    print("====================")

    # Feature set for MERF (keep it smaller + aligned with y_rel)
    X_cols = [
        "y_rel_lag1", "y_rel_lag2",
        "econ_emp_per_1k_lag1", "econ_emp_per_1k_lag2", "d_econ_emp_per_1k_lag1",
        "STATE_% Unemployed_lag1", "STATE_% Unemployed_lag2", "d_STATE_% Unemployed_lag1",
        "state_avg_earnings_lag1", "state_avg_earnings_lag2", "d_state_avg_earnings_lag1",
        "US_hires_per_1k", "t_trend", "post_2020",
    ] + [c for c in health_social if c in df_tr.columns]

    X_cols = [c for c in X_cols if c in df_tr.columns and c in df_te.columns]

    y_tr = df_tr["y_rel"].to_numpy(dtype=float)
    y_te = df_te["y_rel"].to_numpy(dtype=float)

    cl_tr = df_tr["State"].astype(str)
    cl_te = df_te["State"].astype(str)

    # Random effects: intercept + US macro + time trend
    Z_tr = np.column_stack([
        np.ones(len(df_tr), dtype=float),
        df_tr["US_hires_per_1k"].to_numpy(dtype=float),
        df_tr["t_trend"].to_numpy(dtype=float),
    ])
    Z_te = np.column_stack([
        np.ones(len(df_te), dtype=float),
        df_te["US_hires_per_1k"].to_numpy(dtype=float),
        df_te["t_trend"].to_numpy(dtype=float),
    ])

    X_tr_raw = df_tr[X_cols].copy()
    X_te_raw = df_te[X_cols].copy()

    # Walk-forward folds inside TRAIN years
    years_tr = df_tr["Year"].astype(int).to_numpy()
    wf_splits = make_walk_forward_splits_by_year(years_tr, min_train_years=MIN_TRAIN_YEARS_TUNE)
    print("MERF feature count:", len(X_cols))
    print("Walk-forward tuning folds:", len(wf_splits))

    param_grid = {
        "max_depth": [3, 4, 5],                # keep conservative
        "min_samples_leaf": [15, 25, 35],
        "min_samples_split": [30, 50, 80],
        "max_features": [0.4, 0.6, "sqrt"],
    }

    best_rmse = np.inf
    best_params = None

    with HiddenPrints():
        for params in ParameterGrid(param_grid):
            fold_scores = []
            for tr_idx, va_idx in wf_splits:
                imp = SimpleImputer(strategy="median")
                X_f_tr = imp.fit_transform(X_tr_raw.iloc[tr_idx])
                X_f_va = imp.transform(X_tr_raw.iloc[va_idx])

                rf = RandomForestRegressor(
                    n_estimators=TUNE_N_ESTIMATORS,
                    n_jobs=-1,
                    random_state=RANDOM_STATE,
                    **params
                )
                m = MERF(rf, max_iterations=MERF_TUNE_ITERS)
                m.fit(X_f_tr, Z_tr[tr_idx], cl_tr.iloc[tr_idx], y_tr[tr_idx])
                p_va = m.predict(X_f_va, Z_tr[va_idx], cl_tr.iloc[va_idx])
                fold_scores.append(rmse(y_tr[va_idx], p_va))

            avg = float(np.mean(fold_scores))
            if avg < best_rmse:
                best_rmse = avg
                best_params = params

    print("Best CV RMSE (y_rel scale):", round(best_rmse, 5))
    print("Best RF params:", best_params)

    imp_final = SimpleImputer(strategy="median")
    X_tr = imp_final.fit_transform(X_tr_raw)
    X_te = imp_final.transform(X_te_raw)

    rf_final = RandomForestRegressor(
        n_estimators=FINAL_N_ESTIMATORS,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        **(best_params if best_params is not None else {})
    )

    with HiddenPrints():
        merf = MERF(rf_final, max_iterations=MERF_FINAL_ITERS)
        merf.fit(X_tr, Z_tr, cl_tr, y_tr)

    pred_te_rel = merf.predict(X_te, Z_te, cl_te)
    pred_te_abs = reconstruct_hires_from_rel(pred_te_rel, df_te["US_hires_per_1k"].to_numpy(dtype=float))

    print(metrics_block("TEST | MERF(rel->abs)", y_te_abs, pred_te_abs))

    try:
        imp_df = pd.DataFrame({
            "feature": X_cols,
            "importance": merf.trained_fe_model.feature_importances_
        }).sort_values("importance", ascending=False)
        print("\nTop 15 MERF fixed-effect importances:")
        print(imp_df.head(15).to_string(index=False))
    except Exception as e:
        print("MERF importances unavailable:", repr(e))


# ============================================================
# 6) Case-study table for 2023–2024
# ============================================================
case = df_te[["State","Year",TARGET_RAW,"US_hires_per_1k","y_rel","rel_base"]].copy()
case["excess_true_vs_US"] = case[TARGET_RAW] - case["US_hires_per_1k"]
case["excess_true_vs_statebaseline"] = case["y_rel"] - case["rel_base"]
print("\nCase-study table (head):")
print(case.head(10).to_string(index=False))


Rows after lag requirements: 487
Years in model: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Train n=391 | Test n=96
Train years: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Test years : [2023, 2024]

BASELINES (original scale)
{'split': 'TEST | Persistence hires(t-1)', 'r2': 0.3505226337496511, 'rmse': 32.337594145359, 'mae': 25.52804286373968, 'n': 96}
{'split': 'TEST | State shrunk mean y_rel + US', 'r2': 0.7517381795579305, 'rmse': 19.99312418122563, 'mae': 12.787843618838101, 'n': 96}

RESIDUAL RIDGE AR(2) around state baseline (report on original scale)
{'split': 'TEST | Residual Ridge(rel->abs)', 'r2': 0.780223730292301, 'rmse': 18.811182401292037, 'mae': 11.52756520541269, 'n': 96}
Ridge alpha: 38.56620421163472

Top 15 Ridge coefficients by absolute magnitude:
rel_resid_lag1                        0.025830
econ_emp_per_1k_lag1                  0.007767
STATE_Primary Care Physicians Rate   -0.005505
state_avg_earnings_lag2              -0.003091
STATE_% Sm

: 